# 0736 Hard 736 Parse Lisp Expression

In [ ]:
# Time:  O(n^2)
# Space: O(n^2)

class Solution(object):
    def evaluate(self, expression):
        """
        :type expression: str
        :rtype: int
        """
        def getval(lookup, x):
            return lookup.get(x, x)

        def evaluate(tokens, lookup):
            if tokens[0] in ('add', 'mult'):
                a, b = map(int, map(lambda x: getval(lookup, x), tokens[1:]))
                return str(a+b if tokens[0] == 'add' else a*b)
            for i in xrange(1, len(tokens)-1, 2):
                if tokens[i+1]:
                    lookup[tokens[i]] = getval(lookup, tokens[i+1])
            return getval(lookup, tokens[-1])

        tokens, lookup, stk = [''], {}, []
        for c in expression:
            if c == '(':
                if tokens[0] == 'let':
                    evaluate(tokens, lookup)
                stk.append((tokens, dict(lookup)))
                tokens =  ['']
            elif c == ' ':
                tokens.append('')
            elif c == ')':
                val = evaluate(tokens, lookup)
                tokens, lookup = stk.pop()
                tokens[-1] += val
            else:
                tokens[-1] += c
        return int(tokens[0])

# 0741  Hard  741. Cherry Pickup

In [ ]:
class Solution(object):
    def cherryPickup(self, grid):
        if grid[-1][-1] == -1: return 0
        memo, n = {}, len(grid)
        def dp(i1, j1, i2, j2):
            if (i1, j1, i2, j2) in memo: return memo[(i1, j1, i2, j2)]
            if n in (i1, j1, i2, j2) or -1 in (grid[i1][j1], grid[i2][j2]): return -1
            if i1 == i2 == j1 == j2 == n - 1: return grid[-1][-1]
            mx = max(dp(i1+1, j1, i2+1, j2), dp(i1+1, j1, i2, j2+1), dp(i1, j1+1, i2+1, j2), dp(i1, j1+1, i2, j2+1))
            if mx == - 1: out = -1 
            elif i1 == i2 and j1 == j2: out = mx + grid[i1][j1]
            else: out = mx + grid[i1][j1] + grid[i2][j2]
            memo[(i1, j1, i2, j2)] = out
            return out
        return max(0, dp(0, 0, 0, 0))

# 0741 Hard 741 Cherry Pickup

In [ ]:
# Time:  O(n^3)
# Space: O(n^2)

class Solution(object):
    def cherryPickup(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        # dp holds the max # of cherries two k-length paths can pickup.
        # The two k-length paths arrive at (i, k - i) and (j, k - j),
        # respectively.
        n = len(grid)
        dp = [[-1 for _ in xrange(n)] for _ in xrange(n)]
        dp[0][0] = grid[0][0]
        max_len = 2 * (n-1)
        directions = [(0, 0), (-1, 0), (0, -1), (-1, -1)]
        for k in xrange(1, max_len+1):
            for i in reversed(xrange(max(0, k-n+1), min(k+1, n))):  # 0 <= i < n, 0 <= k-i < n
                for j in reversed(xrange(i, min(k+1, n))):          # i <= j < n, 0 <= k-j < n
                    if grid[i][k-i] == -1 or grid[j][k-j] == -1:
                        dp[i][j] = -1
                        continue
                    cnt = grid[i][k-i]
                    if i != j:
                        cnt += grid[j][k-j]
                    max_cnt = -1
                    for direction in directions:
                        ii, jj = i+direction[0], j+direction[1]
                        if ii >= 0 and jj >= 0 and dp[ii][jj] >= 0:
                            max_cnt = max(max_cnt, dp[ii][jj]+cnt)
                    dp[i][j] = max_cnt
        return max(dp[n-1][n-1], 0)

# 0745  Hard  745. Prefix and Suffix Search

In [ ]:
class WordFilter:
    def __init__(self, words):
        self.p, self.s, self.ind = collections.defaultdict(set), collections.defaultdict(set), {}
        for i, w in enumerate(words): 
            self.ind[w] = i
            self.p[""].add(w)
            self.s[""].add(w)
            for i in range(1, len(w) + 1): 
                self.p[w[:i]].add(w)
                self.s[w[-i:]].add(w)
    def f(self, prefix, suffix): return max((self.ind[c] for c in self.p[prefix] & self.s[suffix]), default = -1)

# 0745 Hard 745 Prefix and Suffix Search

In [ ]:
# Time:  ctor:   O(w * l^2), w is the number of words, l is the word length on average
#        search: O(p + s)  , p is the length of the prefix, s is the length of the suffix,
# Space: O(t), t is the number of trie nodes

import collections


class WordFilter(object):

    def __init__(self, words):
        """
        :type words: List[str]
        """
        _trie = lambda: collections.defaultdict(_trie)
        self.__trie = _trie()

        for weight, word in enumerate(words):
            word += '#'
            for i in xrange(len(word)):
                cur = self.__trie
                cur["_weight"] = weight
                for j in xrange(i, 2*len(word)-1):
                    cur = cur[word[j%len(word)]]
                    cur["_weight"] = weight

    def f(self, prefix, suffix):
        """
        :type prefix: str
        :type suffix: str
        :rtype: int
        """
        cur = self.__trie
        for letter in suffix + '#' + prefix:
            if letter not in cur:
                return -1
            cur = cur[letter]
        return cur["_weight"]


# Time:  ctor:   O(w * l), w is the number of words, l is the word length on average
#        search: O(p + s + max(m, n)), p is the length of the prefix, s is the length of the suffix,
#                                      m is the number of the prefix match, n is the number of the suffix match
# Space: O(w * l)
class Trie(object):

    def __init__(self):
        _trie = lambda: collections.defaultdict(_trie)
        self.__trie = _trie()

    def insert(self, word, i):
        def add_word(cur, i):
            if "_words" not in cur:
                cur["_words"] = []
            cur["_words"].append(i)

        cur = self.__trie
        add_word(cur, i)
        for c in word:
            cur = cur[c]
            add_word(cur, i)

    def find(self, word):
        cur = self.__trie
        for c in word:
            if c not in cur:
                return []
            cur = cur[c]
        return cur["_words"]


class WordFilter2(object):

    def __init__(self, words):
        """
        :type words: List[str]
        """
        self.__prefix_trie = Trie()
        self.__suffix_trie = Trie()
        for i in reversed(xrange(len(words))):
            self.__prefix_trie.insert(words[i], i)
            self.__suffix_trie.insert(words[i][::-1], i)

    def f(self, prefix, suffix):
        """
        :type prefix: str
        :type suffix: str
        :rtype: int
        """
        prefix_match = self.__prefix_trie.find(prefix)
        suffix_match = self.__suffix_trie.find(suffix[::-1])
        i, j = 0, 0
        while i != len(prefix_match) and j != len(suffix_match):
            if prefix_match[i] == suffix_match[j]:
                return prefix_match[i]
            elif prefix_match[i] > suffix_match[j]:
                i += 1
            else:
                j += 1
        return -1

# 0749  Hard  749. Contain Virus

In [ ]:
class Solution(object):
    def containVirus(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        if not grid: return None
        N,M=len(grid),len(grid[0])
        
        def around(r,c,t=None):
            # all cells 1-step away from (r,c)
            # optionally, if t!=None, target value must be t
            for d in (-1,1):
                for (rr,cc) in ((r+d,c), (r,c+d)):
                    if 0<=rr<N and 0<=cc<M and (t == None or grid[rr][cc]==t):
                        yield (rr,cc)
        
        def regions():
            regs=[]
            seen=set()
            for r in range(N):
                for c in range(M):
                    if grid[r][c]==1 and (r,c) not in seen:
                        # this is a new region. do a BFS to find all contiguous ones
                        reg=set()
                        front, newfront=[(r,c)], []
                        while front:
                            reg.update(front)
                            while front:
                                (r,c) = front.pop()
                                for (rr,cc) in around(r,c,1):
                                    if (rr, cc) not in reg:
                                        newfront.append((rr,cc))
                                        reg.add((rr, cc))
                            front,newfront=newfront,front
                        regs.append(reg)
                        seen.update(reg)
            return regs
        
        def reg_boundary(reg):
            # cells that would become infected by cells in reg
            bound=set()
            for (r,c) in reg:
                for (rr,cc) in around(r,c,0):
                    bound.add((rr,cc))

            return bound
        
        def reg_walls(reg,bound):
            # number of walls that would need to be built to contain reg
            walls=0
            for (r,c) in bound:
                for (rr,cc) in around(r,c,1):
                    if (rr,cc) in reg:
                        walls+=1    
            return walls
        gwalls=0
        
        while True:
            
            regs=regions()
            
            if not regs: break
            reg = max(regs, key=lambda reg: len(reg_boundary(reg)))
            walls=reg_walls(reg, reg_boundary(reg))
            gwalls+=walls
            
            # neutralize region
            for (r,c) in reg:
                grid[r][c]=2

            # compute next grid iteration after new infections
            infected=set()
            for r in range(N):
                for c in range(M):
                    if grid[r][c]==1:
                        for (rr, cc) in around(r, c, 0):
                            infected.add((rr, cc))

            for r, c in infected:
                grid[r][c]=1

            if not infected: break

        return gwalls

# 0749 Hard 749 Contain Virus

In [ ]:
# Time:  O((m * n)^(4/3)), days = O((m * n)^(1/3))
# Space: O(m * n)

class Solution(object):
    def containVirus(self, grid):
        """
        :type grid: List[List[int]]
        :rtype: int
        """
        directions = [(0, 1), (0, -1), (-1, 0), (1, 0)]

        def dfs(grid, r, c, lookup, regions, frontiers, perimeters):
            if (r, c) in lookup:
                return
            lookup.add((r, c))
            regions[-1].add((r, c))
            for d in directions:
                nr, nc = r+d[0], c+d[1]
                if not (0 <= nr < len(grid) and \
                        0 <= nc < len(grid[r])):
                    continue
                if grid[nr][nc] == 1:
                    dfs(grid, nr, nc, lookup, regions, frontiers, perimeters)
                elif grid[nr][nc] == 0:
                    frontiers[-1].add((nr, nc))
                    perimeters[-1] += 1

        result = 0
        while True:
            lookup, regions, frontiers, perimeters = set(), [], [], []
            for r, row in enumerate(grid):
                for c, val in enumerate(row):
                    if val == 1 and (r, c) not in lookup:
                        regions.append(set())
                        frontiers.append(set())
                        perimeters.append(0)
                        dfs(grid, r, c, lookup, regions, frontiers, perimeters)

            if not regions: break

            triage_idx = frontiers.index(max(frontiers, key = len))
            for i, region in enumerate(regions):
                if i == triage_idx:
                    result += perimeters[i]
                    for r, c in region:
                        grid[r][c] = -1
                    continue
                for r, c in region:
                    for d in directions:
                        nr, nc = r+d[0], c+d[1]
                        if not (0 <= nr < len(grid) and \
                                0 <= nc < len(grid[r])):
                            continue
                        if grid[nr][nc] == 0:
                            grid[nr][nc] = 1

        return result

# 0753  Hard  753. Cracking the Safe

In [ ]:
class Solution:
    def crackSafe(self, n, k):
        s = '0' * (n - 1)
        D = '9876543210'[-k:]
        for _ in range(k**n):
            s += next(d for d in D if (s + d)[-n:] not in s)
        return s

# 0753 Hard 753 Cracking the Safe

In [ ]:
# Time:  O(k^n)
# Space: O(k^n)

class Solution(object):
    def crackSafe(self, n, k):
        """
        :type n: int
        :type k: int
        :rtype: str
        """
        M = k**(n-1)
        P = [q*k+i for i in xrange(k) for q in xrange(M)]  # rotate: i*k^(n-1) + q => q*k + i
        result = [str(k-1)]*(n-1)
        for i in xrange(k**n):
            j = i
            # concatenation in lexicographic order of Lyndon words
            while P[j] >= 0:
                result.append(str(j//M))
                P[j], j = -1, P[j]
        return "".join(result)


# Time:  O(k^n)
# Space: O(k^n)
class Solution2(object):
    def crackSafe(self, n, k):
        """
        :type n: int
        :type k: int
        :rtype: str
        """
        total = k**n
        M = total//k
        unique_rolling_hash = 0
        result = [str(0)]*(n-1)
        lookup = set()
        while len(lookup) < total:
            for i in reversed(xrange(k)):  # preorder like traversal relative to initial result to avoid getting stuck, i.e. don't use 0 until there is no other choice
                new_unique_rolling_hash = unique_rolling_hash*k + i
                if new_unique_rolling_hash not in lookup:
                    lookup.add(new_unique_rolling_hash)
                    result.append(str(i))
                    unique_rolling_hash = new_unique_rolling_hash%M
                    break
        return "".join(result)


# Time:  O(k^n)
# Space: O(k^n)
class Solution3(object):
    def crackSafe(self, n, k):
        """
        :type n: int
        :type k: int
        :rtype: str
        """
        M = k**(n-1)
        def dfs(k, unique_rolling_hash, lookup, result):
            for i in reversed(xrange(k)):  # preorder like traversal relative to initial result to avoid getting stuck, i.e. don't use 0 until there is no other choice
                new_unique_rolling_hash = unique_rolling_hash*k + i
                if new_unique_rolling_hash not in lookup:
                    lookup.add(new_unique_rolling_hash)
                    result.append(str(i))
                    dfs(k, new_unique_rolling_hash%M, lookup, result)
                    break

        unique_rolling_hash = 0
        result = [str(0)]*(n-1)
        lookup = set()
        dfs(k, unique_rolling_hash, lookup, result)
        return "".join(result)


# Time:  O(n * k^n)
# Space: O(n * k^n)
class Solution4(object):
    def crackSafe(self, n, k):
        """
        :type n: int
        :type k: int
        :rtype: str
        """
        result = [str(k-1)]*(n-1)
        lookup = set()
        total = k**n
        while len(lookup) < total:
            node = result[len(result)-n+1:]
            for i in xrange(k):  # preorder like traversal relative to initial result to avoid getting stuck, i.e. don't use k-1 until there is no other choice
                neighbor = "".join(node) + str(i)
                if neighbor not in lookup:
                    lookup.add(neighbor)
                    result.append(str(i))
                    break
        return "".join(result)


# Time:  O(n * k^n)
# Space: O(n * k^n)
class Solution5(object):
    def crackSafe(self, n, k):
        """
        :type n: int
        :type k: int
        :rtype: str
        """
        def dfs(k, node, lookup, result):
            for i in xrange(k):  # preorder like traversal relative to initial result to avoid getting stuck, i.e. don't use k-1 until there is no other choice
                neighbor = node + str(i)
                if neighbor not in lookup:
                    lookup.add(neighbor)
                    result.append(str(i))
                    dfs(k, neighbor[1:], lookup, result)
                    break

        result = [str(k-1)]*(n-1)
        lookup = set()
        dfs(k, "".join(result), lookup, result)
        return "".join(result)

# 0757  Hard  757. Set Intersection Size At Least Two

In [ ]:
class Solution(object):
    def intersectionSizeTwo(self, intervals):
        intervals.sort(key = lambda k: k[1])
        solution = []
        for start, end in intervals:
            if not len(solution) or solution[-1] < start:
                solution.append(end - 1)
                solution.append(end)
            elif solution[-2] < start:
                solution.append(end)
        return len(solution)

# 0757 Hard 757 Set Intersection Size At Least Two

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

class Solution(object):
    def intersectionSizeTwo(self, intervals):
        """
        :type intervals: List[List[int]]
        :rtype: int
        """
        intervals.sort(key = lambda s_e: (s_e[0], -s_e[1]))
        cnts = [2] * len(intervals)
        result = 0
        while intervals:
            (start, _), cnt = intervals.pop(), cnts.pop()
            for s in xrange(start, start+cnt):
                for i in xrange(len(intervals)):
                    if cnts[i] and s <= intervals[i][1]:
                        cnts[i] -= 1
            result += cnt
        return result

# 0759  Hard  759. Employee Free Time

In [ ]:
# Definition for an interval.
# class Interval:
#     def __init__(self, s=0, e=0):
#         self.start = s
#         self.end = e

class Solution:
    def employeeFreeTime(self, schedule):
        intervals = sorted(((intr.start, intr.end) for emp in schedule for intr in emp), key = lambda x: x[0])
        res, stack = [], []
        for s, e in intervals:
            if not stack:
                stack.append((s, e))
            elif s <= stack[-1][-1]:
                stack.append((s, max(e, stack.pop()[1])))
            else:
                res.append([stack[-1][1], s])
                stack.append((s, e))
        return res

# 0759 Hard 759 Employee Free Time

In [ ]:
# Time:  O(m * logn), m is the number of schedule, n is the number of employees, m >= n
# Space: O(n)

import heapq


class Interval(object):
    def __init__(self, s=0, e=0):
        self.start = s
        self.end = e


class Solution(object):
    def employeeFreeTime(self, schedule):
        """
        :type schedule: List[List[Interval]]
        :rtype: List[Interval]
        """
        result = []
        min_heap = [(emp[0].start, eid, 0) for eid, emp in enumerate(schedule)]
        heapq.heapify(min_heap)
        last_end = -1
        while min_heap:
            t, eid, i = heapq.heappop(min_heap)
            if 0 <= last_end < t:
                result.append(Interval(last_end, t))
            last_end = max(last_end, schedule[eid][i].end)
            if i+1 < len(schedule[eid]):
                heapq.heappush(min_heap, (schedule[eid][i+1].start, eid, i+1))
        return result

# 0761  Hard  761. Special Binary String

In [ ]:
class Solution:
    def makeLargestSpecial(self, S: str) -> str:
        count = i = 0
        res = []
        for j, v in enumerate(S):
            count = count + 1 if v=='1' else count - 1
            if count == 0:
                res.append('1' + self.makeLargestSpecial(S[i + 1:j]) + '0')
                i = j + 1
        return ''.join(sorted(res)[::-1])

# 0761 Hard 761 Special Binary String

In [ ]:
# Time:  f(n) = k * f(n/k) + n/k * klogk <= O(logn * nlogk) <= O(n^2)
#        n is the length of S, k is the max number of special strings in each depth
# Space: O(n)

class Solution(object):
    def makeLargestSpecial(self, S):
        """
        :type S: str
        :rtype: str
        """
        result = []
        anchor = count = 0
        for i, v in enumerate(S):
            count += 1 if v == '1' else -1
            if count == 0:
                result.append("1{}0".format(self.makeLargestSpecial(S[anchor+1:i])))
                anchor = i+1
        result.sort(reverse = True)
        return "".join(result)

# 0765  Hard  765. Couples Holding Hands

In [ ]:
class Solution:
    def minSwapsCouples(self, row):
        res, index = 0, {num: i for i, num in enumerate(row)}
        for i in range(0, len(row), 2):
            if row[i] % 2 == 0 and row[i + 1] != row[i] + 1: 
                f = row[i + 1]
                row[i + 1], row[index[row[i] + 1]] = row[i] + 1, row[i + 1]
                index[row[i] + 1], index[f] = i + 1, index[row[i] + 1]
                res += 1
            elif row[i] % 2 != 0 and row[i + 1] != row[i] - 1:
                f = row[i + 1]
                row[i + 1], row[index[row[i] - 1]], index[row[i + 1]] = row[i] - 1, row[i + 1], index[row[i] - 1]
                index[row[i] - 1], index[f] = i + 1, index[row[i] - 1]
                res += 1
        return res

# 0765 Hard 765 Couples Holding Hands

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def minSwapsCouples(self, row):
        """
        :type row: List[int]
        :rtype: int
        """
        N = len(row)//2
        couples = [[] for _ in xrange(N)]
        for seat, num in enumerate(row):
            couples[num//2].append(seat//2)
        adj = [[] for _ in xrange(N)]
        for couch1, couch2 in couples:
            adj[couch1].append(couch2)
            adj[couch2].append(couch1)

        result = 0
        for couch in xrange(N):
            if not adj[couch]: continue
            couch1, couch2 = couch, adj[couch].pop()
            while couch2 != couch:
                result += 1
                adj[couch2].remove(couch1)
                couch1, couch2 = couch2, adj[couch2].pop()
        return result  # also equals to N - (# of cycles)

# 0768  Hard  768. Max Chunks To Make Sorted II

In [ ]:
class Solution:
    def maxChunksToSorted(self, arr):
        mx, mn, res, check = 0, 10 ** 9, 0, [[0, 0] for _ in range(len(arr))]
        for i in range(len(arr)):
            if arr[i] > mx: mx = arr[i]
            check[i][0] = mx
        for i in range(len(arr) -1, -1, -1):
            check[i][1] = mn
            if arr[i] < mn: mn = arr[i]
        for c in check:
            if c[0] <= c[1]: res += 1
        return res

# 0768 Hard 768 Max Chunks To Make Sorted II

In [ ]:
# Time:  O(n)
# Space: O(n)

# mono stack solution
class Solution(object):
    def maxChunksToSorted(self, arr):
        """
        :type arr: List[int]
        :rtype: int
        """
        result, increasing_stk = 0, []
        for num in arr:
            max_num = num if not increasing_stk else max(increasing_stk[-1], num)
            while increasing_stk and increasing_stk[-1] > num:
                increasing_stk.pop()
            increasing_stk.append(max_num)
        return len(increasing_stk)


# Time:  O(nlogn)
# Space: O(n)
class Solution2(object):
    def maxChunksToSorted(self, arr):
        """
        :type arr: List[int]
        :rtype: int
        """
        def compare(i1, i2):
            return arr[i1]-arr[i2] if arr[i1] != arr[i2] else i1-i2

        idxs = [i for i in xrange(len(arr))]
        result, max_i = 0, 0
        for i, v in enumerate(sorted(idxs, cmp=compare)):
            max_i = max(max_i, v)
            if max_i == i:
                result += 1
        return result

# 0770  Hard  770. Basic Calculator IV

In [ ]:
class Solution(object):
    def basicCalculatorIV(self, s, evalvars, evalints):
        s.strip()
        d = dict(zip(evalvars, evalints))
        s = s.replace(' ', '')
        ts = re.findall('\u005Cd+|[-()+*]|[^-()+*]+', s)
        
        def add(p, q):
            i, j = 0, 0
            r = []
            while i < len(p) and j < len(q):
                v, c = p[i]
                v2, c2 = q[j]
                if v == v2:
                    if c + c2 != 0:
                        r.append((v, c + c2))
                    i += 1
                    j += 1
                elif len(v) > len(v2) or len(v) == len(v2) and v < v2:
                    r.append(p[i])
                    i += 1
                else:
                    r.append(q[j])
                    j += 1
                            
            r += p[i:]
            r += q[j:]
            return r

        def neg(p):
            r = []
            for v, c in p:
                r.append((v, -c))
            return r

        def sub(p, q):
            return add(p, neg(q))

        def mult(p, q):
            r = []
            for v, c in p:
                for v2, c2 in q:
                    r = add(r, [(sorted(v + v2), c * c2)])
            return r
            
        def prec(c):
            return 0 if c in [')'] else 1 if c in ['+', '-'] else 2
            
        i = 0 
        def expr(p):
            nonlocal i, ts
            if ts[i] == '(':
                i += 1
                v = expr(0)
                i += 1
            elif ts[i] == '-':
                i += 1
                v = neg(expr(3))
            elif re.match('\u005Cd+', ts[i]):
                if ts[i] != '0':
                    v = [([], int(ts[i]))]
                else:
                    v = []
            else:
                if ts[i] in d:
                    if d[ts[i]] != 0:
                        v = [([], d[ts[i]])]
                    else:
                        v  = []
                else:
                    v = [([ts[i]], 1)]
            while i < len(ts) - 2 and prec(ts[i+1]) > p:
                op = ts[i+1]
                i += 2
                v2 = expr(prec(op))
                if op == '+': v = add(v, v2)
                if op == '-': v = sub(v, v2)
                if op == '*': v = mult(v, v2)
                
            return v

        def tostrings(p):
            r = []
            for v, c in p:
                if v == []:
                    r.append(str(c))
                else:
                    r.append(str(c) + '*' + '*'.join(v))
            return r
        
        return tostrings(expr(0))

# 0770 Hard 770 Basic Calculator IV

In [ ]:
# Time:  +:        O(d * t), t is the number of terms,
#                            d is the average degree of terms
#        -:        O(d * t)
#        *:        O(d * t^2)
#        eval:     O(d * t)
#        to_list:  O(d * tlogt)
# Space: O(e + d * t), e is the number of evalvars

import collections
import itertools
import operator


def clear(result):
    to_remove = [k for k, v in result.iteritems() if v == 0]
    for k in to_remove:
        result.pop(k)


class Poly(collections.Counter):
    def __init__(self, expr=None):
        if expr is None:
            return
        if expr.isdigit():
            if int(expr):
                self.update({(): int(expr)})
        else:
            self[(expr,)] += 1

    def __add__(self, other):
        result = Poly()
        result.update(self)
        result.update(other)
        clear(result)
        return result

    def __sub__(self, other):
        result = Poly()
        result.update(self)
        result.update({k: -v for k, v in other.iteritems()})
        clear(result)
        return result

    def __mul__(self, other):
        def merge(k1, k2):
            result = []
            i, j = 0, 0
            while i != len(k1) or j != len(k2):
                if j == len(k2) or (i != len(k1) and k1[i] < k2[j]):
                    result.append(k1[i])
                    i += 1
                else:
                    result.append(k2[j])
                    j += 1
            return result

        result = Poly()
        for k1, v1 in self.iteritems():
            for k2, v2 in other.iteritems():
                result.update({tuple(merge(k1, k2)): v1*v2})
        clear(result)
        return result

    def eval(self, lookup):
        result = Poly()
        for polies, c in self.iteritems():
            key = []
            for var in polies:
                if var in lookup:
                    c *= lookup[var]
                else:
                    key.append(var)
            result[tuple(key)] += c
        clear(result)
        return result

    def to_list(self):
        return ["*".join((str(v),) + k)
                for k, v in sorted(self.iteritems(),
                                   key=lambda x: (-len(x[0]), x[0]))]


class Solution(object):
    def basicCalculatorIV(self, expression, evalvars, evalints):
        """
        :type expression: str
        :type evalvars: List[str]
        :type evalints: List[int]
        :rtype: List[str]
        """
        ops = {'+':operator.add, '-':operator.sub, '*':operator.mul}
        def compute(operands, operators):
            right, left = operands.pop(), operands.pop()
            operands.append(ops[operators.pop()](left, right))

        def parse(s):
            precedence = {'+':0, '-':0, '*':1}
            operands, operators, operand = [], [], []
            for i in xrange(len(s)):
                if s[i].isalnum():
                    operand.append(s[i])
                    if i == len(s)-1 or not s[i+1].isalnum():
                        operands.append(Poly("".join(operand)))
                        operand = []
                elif s[i] == '(':
                    operators.append(s[i])
                elif s[i] == ')':
                    while operators[-1] != '(':
                        compute(operands, operators)
                    operators.pop()
                elif s[i] in precedence:
                    while operators and operators[-1] in precedence and \
                          precedence[operators[-1]] >= precedence[s[i]]:
                        compute(operands, operators)
                    operators.append(s[i])
            while operators:
                compute(operands, operators)
            return operands[-1]

        lookup = dict(itertools.izip(evalvars, evalints))
        return parse(expression).eval(lookup).to_list()


class Solution2(object):
    def basicCalculatorIV(self, expression, evalvars, evalints):
        """
        :type expression: str
        :type evalvars: List[str]
        :type evalints: List[int]
        :rtype: List[str]
        """
        def compute(operands, operators):
            left, right = operands.pop(), operands.pop()
            op = operators.pop()
            if op == '+':
                operands.append(left + right)
            elif op == '-':
                operands.append(left - right)
            elif op == '*':
                operands.append(left * right)

        def parse(s):
            if not s:
                return Poly()
            operands, operators = [], []
            operand = ""
            for i in reversed(xrange(len(s))):
                if s[i].isalnum():
                    operand += s[i]
                    if i == 0 or not s[i-1].isalnum():
                        operands.append(Poly(operand[::-1]))
                        operand = ""
                elif s[i] == ')' or s[i] == '*':
                    operators.append(s[i])
                elif s[i] == '+' or s[i] == '-':
                    while operators and operators[-1] == '*':
                        compute(operands, operators)
                    operators.append(s[i])
                elif s[i] == '(':
                    while operators[-1] != ')':
                        compute(operands, operators)
                    operators.pop()
            while operators:
                compute(operands, operators)
            return operands[-1]

        lookup = dict(itertools.izip(evalvars, evalints))
        return parse(expression).eval(lookup).to_list()

# 0772  Hard  772. Basic Calculator III

In [ ]:
class Solution:
    def calculate(self, s: str) -> int:
        def calc(n2, op, n1): 
            return n1 * n2 if op == '*' else n1 // n2 if op == '/' else n1 + n2 if op == '+' else n1 - n2
        def calc2(arr):
            if len(arr) == 1:
                return arr.pop()
            res = arr[0]
            for j in range(2, len(arr), 2):
                res = calc(arr[j], arr[j - 1], res)
            return res
        stack, i, num = [], 0, 0
        while i < len(s):
            j = i
            while j < len(s) and s[j].isdigit():
                num, j = num * 10 + int(s[j]), j + 1
            if i != j:
                stack.append(calc(num, stack.pop(), stack.pop()) if stack and stack[-1] in "*/" else num)
                num, j = 0, j - 1
            elif s[i] == ")":
                ind = len(stack) - stack[::-1].index('(') - 1
                stack[ind:] = [calc2(stack[ind + 1:])]
                if len(stack) > 1 and stack[-2] in '*/':
                    stack.append(calc(stack.pop(), stack.pop(), stack.pop()))
            elif s[i] != ' ':
                stack.append(s[i])
            i = j + 1
        return calc2(stack)

# 0772 Hard 772 Basic Calculator III

In [ ]:
# Time:  O(n)
# Space: O(n)

import operator


class Solution(object):
    def calculate(self, s):
        """
        :type s: str
        :rtype: int
        """
        def compute(operands, operators):
            right, left = operands.pop(), operands.pop()
            operands.append(ops[operators.pop()](left, right))

        ops = {'+':operator.add, '-':operator.sub, '*':operator.mul, '/':operator.div}
        precedence = {'+':0, '-':0, '*':1, '/':1}
        operands, operators, operand = [], [], 0
        for i in xrange(len(s)):
            if s[i].isdigit():
                operand = operand*10 + int(s[i])
                if i == len(s)-1 or not s[i+1].isdigit():
                    operands.append(operand)
                    operand = 0
            elif s[i] == '(':
                operators.append(s[i])
            elif s[i] == ')':
                while operators[-1] != '(':
                    compute(operands, operators)
                operators.pop()
            elif s[i] in precedence:
                while operators and operators[-1] in precedence and \
                      precedence[operators[-1]] >= precedence[s[i]]:
                    compute(operands, operators)
                operators.append(s[i])
        while operators:
            compute(operands, operators)
        return operands[-1]


# Time:  O(n)
# Space: O(n)
class Solution2(object):
    def calculate(self, s):
        """
        :type s: str
        :rtype: int
        """
        operands, operators = [], []
        operand = ""
        for i in reversed(xrange(len(s))):
            if s[i].isdigit():
                operand += s[i]
                if i == 0 or not s[i-1].isdigit():
                    operands.append(int(operand[::-1]))
                    operand = ""
            elif s[i] == ')' or s[i] == '*' or s[i] == '/':
                operators.append(s[i])
            elif s[i] == '+' or s[i] == '-':
                while operators and \
                      (operators[-1] == '*' or operators[-1] == '/'):
                    self.compute(operands, operators)
                operators.append(s[i])
            elif s[i] == '(':
                while operators[-1] != ')':
                    self.compute(operands, operators)
                operators.pop()

        while operators:
            self.compute(operands, operators)

        return operands[-1]

    def compute(self, operands, operators):
        left, right = operands.pop(), operands.pop()
        op = operators.pop()
        if op == '+':
            operands.append(left + right)
        elif op == '-':
            operands.append(left - right)
        elif op == '*':
            operands.append(left * right)
        elif op == '/':
            operands.append(left / right)

# 0773  Hard  773. Sliding Puzzle

In [ ]:
class Solution:
    def slidingPuzzle(self, board):
        moves, used, cnt = {0: {1, 3}, 1:{0, 2, 4}, 2:{1, 5}, 3:{0, 4}, 4:{1, 3, 5}, 5:{2, 4}}, set(), 0
        s = "".join(str(c) for row in board for c in row)
        q = [(s, s.index("0"))]
        while q:
            new = []
            for s, i in q:
                used.add(s)
                if s == "123450":
                    return cnt
                arr = [c for c in s]
                for move in moves[i]:
                    new_arr = arr[:]
                    new_arr[i], new_arr[move] = new_arr[move], new_arr[i]
                    new_s = "".join(new_arr)
                    if new_s not in used:
                        new.append((new_s, move))
            cnt += 1
            q = new
        return -1

# 0773 Hard 773 Sliding Puzzle

In [ ]:
# Time:  O((m * n) * (m * n)!)
# Space: O((m * n) * (m * n)!)

import heapq
import itertools


# A* Search Algorithm
class Solution(object):
    def slidingPuzzle(self, board):
        """
        :type board: List[List[int]]
        :rtype: int
        """
        def dot(p1, p2):
            return p1[0]*p2[0]+p1[1]*p2[1]

        def heuristic_estimate(board, R, C, expected):
            result = 0
            for i in xrange(R):
                for j in xrange(C):
                    val = board[C*i + j]
                    if val == 0: continue
                    r, c = expected[val]
                    result += abs(r-i) + abs(c-j)
            return result

        R, C = len(board), len(board[0])
        begin = tuple(itertools.chain(*board))
        end = tuple(range(1, R*C) + [0])
        expected = {(C*i+j+1) % (R*C) : (i, j)
                    for i in xrange(R) for j in xrange(C)}

        min_steps = heuristic_estimate(begin, R, C, expected)
        closer, detour = [(begin.index(0), begin)], []
        lookup = set()
        while True:
            if not closer:
                if not detour:
                    return -1
                min_steps += 2
                closer, detour = detour, closer
            zero, board = closer.pop()
            if board == end:
                return min_steps
            if board not in lookup:
                lookup.add(board)
                r, c = divmod(zero, C)
                for direction in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                    i, j = r+direction[0], c+direction[1]
                    if 0 <= i < R and 0 <= j < C:
                        new_zero = i*C+j
                        tmp = list(board)
                        tmp[zero], tmp[new_zero] = tmp[new_zero], tmp[zero]
                        new_board = tuple(tmp)
                        r2, c2 = expected[board[new_zero]]
                        r1, c1 = divmod(zero, C)
                        r0, c0 = divmod(new_zero, C)
                        is_closer = dot((r1-r0, c1-c0), (r2-r0, c2-c0)) > 0
                        (closer if is_closer else detour).append((new_zero, new_board))
        return min_steps


# Time:  O((m * n) * (m * n)! * log((m * n)!))
# Space: O((m * n) * (m * n)!)
# A* Search Algorithm
class Solution2(object):
    def slidingPuzzle(self, board):
        """
        :type board: List[List[int]]
        :rtype: int
        """
        def heuristic_estimate(board, R, C, expected):
            result = 0
            for i in xrange(R):
                for j in xrange(C):
                    val = board[C*i + j]
                    if val == 0: continue
                    r, c = expected[val]
                    result += abs(r-i) + abs(c-j)
            return result

        R, C = len(board), len(board[0])
        begin = tuple(itertools.chain(*board))
        end = tuple(range(1, R*C) + [0])
        end_wrong = tuple(range(1, R*C-2) + [R*C-1, R*C-2, 0])
        expected = {(C*i+j+1) % (R*C) : (i, j)
                    for i in xrange(R) for j in xrange(C)}

        min_heap = [(0, 0, begin.index(0), begin)]
        lookup = {begin: 0}
        while min_heap:
            f, g, zero, board = heapq.heappop(min_heap)
            if board == end: return g
            if board == end_wrong: return -1
            if f > lookup[board]: continue

            r, c = divmod(zero, C)
            for direction in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                i, j = r+direction[0], c+direction[1]
                if 0 <= i < R and 0 <= j < C:
                    new_zero = C*i+j
                    tmp = list(board)
                    tmp[zero], tmp[new_zero] = tmp[new_zero], tmp[zero]
                    new_board = tuple(tmp)
                    f = g+1+heuristic_estimate(new_board, R, C, expected)
                    if f < lookup.get(new_board, float("inf")):
                        lookup[new_board] = f
                        heapq.heappush(min_heap, (f, g+1, new_zero, new_board))
        return -1

# 0774  Hard  774. Minimize Max Distance to Gas Station

In [ ]:
class Solution:
    def minmaxGasDist(self, st, K):
        left, right = 1e-6, st[-1] - st[0]
        while left + 1e-6 < right:
            mid = (left + right) / 2
            count = 0
            for a, b in zip(st, st[1:]):
                count += math.ceil((b - a) / mid) - 1
            if count > K:
                left = mid
            else:
                right = mid
        return right

# 0774 Hard 774 Minimize Max Distance to Gas Station

In [ ]:
# Time:  O(nlogr)
# Space: O(1)

class Solution(object):
    def minmaxGasDist(self, stations, K):
        """
        :type stations: List[int]
        :type K: int
        :rtype: float
        """
        def possible(stations, K, guess):
            return sum(int((stations[i+1]-stations[i]) / guess)
                       for i in xrange(len(stations)-1)) <= K

        left, right = 0, 10**8
        while right-left > 1e-6:
            mid = left + (right-left)/2.0
            if possible(mid):
                right = mid
            else:
                left = mid
        return left

# 0778  Hard  778. Swim in Rising Water

In [ ]:
class Solution:
    def swimInWater(self, grid):
        heap, res, n, visited = [(grid[0][0], 0, 0)], 0, len(grid), set()
        while True:
            d, i, j = heapq.heappop(heap)
            if d > res: res = d
            if i == j == n - 1: return res
            for x, y in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)):
                if 0 <= x < n and 0 <= y < n and (x, y) not in visited: 
                    visited.add((x, y))
                    heapq.heappush(heap, (grid[x][y], x, y))

# 0778 Hard 778 Swim in Rising Water

In [ ]:
"""
Starting from `(0, 0)` we keep on selecting the neighbor with lower elevation until we reach the end.
There are four possible neighbors `(i+1, j), (i-1, j), (i, j+1), (i, j-1)`.
Every time we select neighbor  we check if we visited and choose the smallest elevation to go.
"""
#Wrong
class Solution(object):
    def swimInWater(self, grid):
        def getNext(i, j):
            options = [(i+1, j), (i-1, j), (i, j+1), (i, j-1)]
            elevation = float('inf')
            opt = (None, None)
            for i_next, j_next in options:
                if i_next<0 or i_next>=N or j_next<0 or j_next>=M: continue
                if (i_next, j_next) in visited: continue
                if grid[i_next][j_next]<elevation:
                    elevation = grid[i_next][j_next]
                    opt = (i_next, j_next)
            return opt

        N = len(grid)
        M = len(grid[0])
        ans = grid[-1][-1]
        visited = set()
        i = j = 0

        while i<N-1 and j<M-1:
            print i, j
            visited.add((i, j))
            ans = max(ans, grid[i][j])
            i, j = getNext(i, j)
        return ans

"""
The above solution is worng, because we might reach a point which all its neighbor is visited (dead end).
That is why we need a heap to get the next posible point to go.
And when we reach dead end, we pop out the next avaliable option.
The time complexity is O((N^2)*LogN), for there are N^2 point and every heap operation for it is LogN.
Space complexity is O(N^2)
"""
#Heap solution
class Solution(object):
    def swimInWater(self, grid):
        ans = grid[-1][-1]
        N = len(grid)
        pq = []
        seen = set()

        heapq.heappush(pq, (grid[0][0], 0, 0))
        while pq:
            t, i, j = heapq.heappop(pq)
            ans = max(ans, t)
            if i==N-1 and j==N-1: return ans
            for i_next, j_next in [(i+1, j), (i-1, j), (i, j+1), (i, j-1)]:
                if i_next<0 or i_next>=N or j_next<0 or j_next>=N: continue
                if (i_next, j_next) in seen: continue
                heapq.heappush(pq, (grid[i_next][j_next], i_next, j_next))
                seen.add((i_next, j_next))
        return ans


"""
The answer must lie between `l` and `h`.
Where h is the value we sure that it can pass, l is the value it might or might not pass.
So we gradually test the value between `l~h` by binary search.
Until we find the value which is the lowest possible time that can pass. #[0]

I init the `l` with the `t` of the destination, because we couldn't have been reach the destination without using `t` amount of time.
I init the `h` with the max `t` in the entire grid, since we can swim withim this time no matter what.

The function `canPassWtihTimeLimit(t)` takes a parameter `t` and use DFS to see if we can swim to the destination in the time limit t.

The time complexity is O((N^2)*LogN).
Find the max in the grid took O(N^2).
`canPassWtihTimeLimit(t)` took O(N^2), because we might possibly travel the entire grid.
We call `canPassWtihTimeLimit(t)` about O(LogN) of time because we use the binary search concept to navigate the `l` and `r`.
"""
class Solution(object):
    def swimInWater(self, grid):
        def canPassWtihTimeLimit(t):
            stack = []
            seen = set()

            if grid[0][0]<=t: stack.append((0, 0))
            while stack:
                i, j = stack.pop()
                if i==N-1 and j==N-1: return True
                seen.add((i, j))
                for i_next, j_next in [(i+1, j), (i-1, j), (i, j+1), (i, j-1)]:
                    if i_next<0 or i_next>=N or j_next<0 or j_next>=N: continue
                    if (i_next, j_next) in seen: continue
                    if grid[i_next][j_next]>t: continue
                    stack.append((i_next, j_next))
            return False

        N = len(grid)
        l = grid[-1][-1]
        h = max(map(max, grid)) #get max value in the grid
        while True:
            m = (l+h)/2
            p = canPassWtihTimeLimit(m)
            if p and not canPassWtihTimeLimit(m-1): return m #[0]
            if p:
                h = m
            else:
                l = m+1
        return h

# 0780  Hard  780. Reaching Points

In [ ]:
class Solution:
    def reachingPoints(self, sx, sy, tx, ty):
        while sx<tx and sy<ty: tx,ty = tx%ty,ty%tx
        return sx==tx and (ty-sy)%sx==0 or sy==ty and (tx-sx)%sy==0

# 0780 Hard 780 Reaching Points

In [ ]:
# Time:  O(log(max(m, n)))
# Space: O(1)

class Solution(object):
    def reachingPoints(self, sx, sy, tx, ty):
        """
        :type sx: int
        :type sy: int
        :type tx: int
        :type ty: int
        :rtype: bool
        """
        while tx >= sx and ty >= sy:
            if tx < ty:
                sx, sy = sy, sx
                tx, ty = ty, tx
            if ty > sy:
                tx %= ty
            else:
                return (tx - sx) % ty == 0

        return False

# 0782  Hard  782. Transform to Chessboard

In [ ]:
class Solution:
    def movesToChessboard(self, b):
        N = len(b)
        if any(b[0][0] ^ b[i][0] ^ b[0][j] ^ b[i][j] for i in range(N) for j in range(N)): return -1
        if not N // 2 <= sum(b[0]) <= (N + 1) // 2: return -1
        if not N // 2 <= sum(b[i][0] for i in range(N)) <= (N + 1) // 2: return -1
        col = sum(b[0][i] == i % 2 for i in range(N))
        row = sum(b[i][0] == i % 2 for i in range(N))
        if N % 2:
            if col % 2: col = [col, N - col][col % 2]
            if row % 2: row = N - row
        else:
            col = min(N - col, col)
            row = min(N - row, row)
        return (col + row) // 2

# 0782 Hard 782 Transform to Chessboard

In [ ]:
# Time:  O(n^2)
# Space: O(n^2), used by Counter, this could be reduced to O(n) by skipping invalid input

import collections
import itertools


class Solution(object):
    def movesToChessboard(self, board):
        """
        :type board: List[List[int]]
        :rtype: int
        """
        N = len(board)
        result = 0
        for count in (collections.Counter(map(tuple, board)), \
                      collections.Counter(itertools.izip(*board))):
            if len(count) != 2 or \
               sorted(count.values()) != [N/2, (N+1)/2]:
                return -1

            seq1, seq2 = count
            if any(x == y for x, y in itertools.izip(seq1, seq2)):
                return -1
            begins = [int(seq1.count(1) * 2 > N)] if N%2 else [0, 1]
            result += min(sum(int(i%2 != v) for i, v in enumerate(seq1, begin)) \
                          for begin in begins) / 2
        return result

# 0793  Hard  793. Preimage Size of Factorial Zeroes Function

In [ ]:
class Solution:
    
    def count(self, num):
        cnt = 0
        while num:
            cnt += num // 5
            num //= 5
        return cnt 
    
    def preimageSizeFZF(self, K):
        l, r = 0, 2 ** 63 - 1
        while l < r:
            mid = (l + r) // 2
            if self.count(mid) < K:
                l = mid + 1
            else:
                r = mid
        return 5 - (l % 5) if self.count(l) == K else 0

# 0793 Hard 793 Preimage Size of Factorial Zeroes Function

In [ ]:
# Time:  O((logn)^2)
# Space: O(1)

class Solution(object):
    def preimageSizeFZF(self, K):
        """
        :type K: int
        :rtype: int
        """
        def count_of_factorial_primes(n, p):
            cnt = 0
            while n > 0:
                cnt += n//p
                n //= p
            return cnt

        p = 5
        left, right = 0, p*K
        while left <= right:
            mid = left + (right-left)//2
            if count_of_factorial_primes(mid, p) >= K:
                right = mid-1
            else:
                left = mid+1
        return p if count_of_factorial_primes(left, p) == K else 0

# 0798  Hard  798. Smallest Rotation with Highest Score

In [ ]:
class Solution:
    def bestRotation(self, A):
        N = len(A)
        change = [1] * N
        for i in range(N): change[(i - A[i] + 1) % N] -= 1
        for i in range(1, N): change[i] += change[i - 1]
        return change.index(max(change))

# 0798 Hard 798 Smallest Rotation with Highest Score

In [ ]:
# Time:  O(n)
# Space: O(n)

class Solution(object):
    def bestRotation(self, A):
        """
        :type A: List[int]
        :rtype: int
        """
        N = len(A)
        change = [1] * N
        for i in xrange(N):
            change[(i-A[i]+1)%N] -= 1
        for i in xrange(1, N):
            change[i] += change[i-1]
        return change.index(max(change))

# 0801 Hard 801 Minimum Swaps To Make Sequences Increasing

In [ ]:
class Solution(object):
    def minSwap(self, A, B):
        keep = [float('inf') for _ in xrange(len(A))]
        swap = [float('inf') for _ in xrange(len(A))]
        
        keep[0] = 0
        swap[0] = 1
        
        for i in xrange(1, len(A)):
            
            if A[i]>A[i-1] and B[i]>B[i-1]:
                keep[i] = keep[i-1]
                swap[i] = swap[i-1]+1
                
            if A[i]>B[i-1] and B[i]>A[i-1]:
                keep[i] = min(keep[i], swap[i-1])
                swap[i] = min(swap[i], keep[i-1]+1)
                
        return min(keep[-1], swap[-1])

# 0803  Hard  803. Bricks Falling When Hit

In [ ]:
class Solution:
    def hitBricks(self, grid, hits):
        m, n, ret = len(grid), len(grid[0]), [0]*len(hits)
        # Connect unconnected bricks and 
        def dfs(i, j):
            if not (0 <= i <m and 0 <= j <n) or grid[i][j] != 1:
                return 0
            grid[i][j] = 2
            return 1 + sum(dfs(x, y) for x, y in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)))
        # Check whether (i, j) is connected to Not Falling Bricks
        def is_connected(i, j):
            return not i or any(0 <= x < m and 0 <= y < n and grid[x][y] == 2 for x, y in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)))
        # Mark whether there is a brick at the each hit
        for i, j in hits:
            grid[i][j] -= 1      
        # Get grid after all hits
        for i in range(n):
            dfs(0, i)
        # Reversely add the block of each hits and get count of newly add bricks
        for k in reversed(range(len(hits))):
            i, j = hits[k]
            grid[i][j] += 1
            if grid[i][j] and is_connected(i, j):
                ret[k] = dfs(i, j) - 1
        return ret

# 0803 Hard 803 Bricks Falling When Hit

In [ ]:
# Time:  O(r * c)
# Space: O(r * c)

class UnionFind(object):
    def __init__(self, n):
        self.set = range(n+1)
        self.size = [1]*(n+1)
        self.size[-1] = 0

    def find_set(self, x):
        if self.set[x] != x:
            self.set[x] = self.find_set(self.set[x])  # path compression.
        return self.set[x]

    def union_set(self, x, y):
        x_root, y_root = map(self.find_set, (x, y))
        if x_root == y_root:
            return False
        self.set[min(x_root, y_root)] = max(x_root, y_root)
        self.size[max(x_root, y_root)] += self.size[min(x_root, y_root)]
        return True

    def top(self):
        return self.size[self.find_set(len(self.size)-1)]


class Solution(object):
    def hitBricks(self, grid, hits):
        """
        :type grid: List[List[int]]
        :type hits: List[List[int]]
        :rtype: List[int]
        """
        def index(C, r, c):
            return r*C+c

        directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]
        R, C = len(grid), len(grid[0])

        hit_grid = [row[:] for row in grid]
        for i, j in hits:
            hit_grid[i][j] = 0

        union_find = UnionFind(R*C)
        for r, row in enumerate(hit_grid):
            for c, val in enumerate(row):
                if not val:
                    continue
                if r == 0:
                    union_find.union_set(index(C, r, c), R*C)
                if r and hit_grid[r-1][c]:
                    union_find.union_set(index(C, r, c), index(C, r-1, c))
                if c and hit_grid[r][c-1]:
                    union_find.union_set(index(C, r, c), index(C, r, c-1))

        result = []
        for r, c in reversed(hits):
            prev_roof = union_find.top()
            if grid[r][c] == 0:
                result.append(0)
                continue
            for d in directions:
                nr, nc = (r+d[0], c+d[1])
                if 0 <= nr < R and 0 <= nc < C and hit_grid[nr][nc]:
                    union_find.union_set(index(C, r, c), index(C, nr, nc))
            if r == 0:
                union_find.union_set(index(C, r, c), R*C)
            hit_grid[r][c] = 1
            result.append(max(0, union_find.top()-prev_roof-1))
        return result[::-1]

# 0805  Hard  805. Split Array With Same Average

In [ ]:
class Solution:
    def splitArraySameAverage(self, A):
        def find(target, k, i):
            if (target,k) in not_found and not_found[(target,k)] <= i: return False
            if k == 0: return target == 0
            if k + i > len(A): return False
            res = find(target - A[i], k - 1, i + 1) or find(target, k, i + 1)
            if not res: not_found[(target, k)] = min(not_found.get((target, k), n), i)
            return res
        not_found = dict()
        n, s = len(A), sum(A)
        return any(find(s * i / n, i, 0) for i in range(1, n // 2 + 1) if s * i % n == 0)

# 0805 Hard 805 Split Array With Same Average

In [ ]:
# Time:  O(n^4)
# Space: O(n^3)

class Solution(object):
    def splitArraySameAverage(self, A):
        """
        :type A: List[int]
        :rtype: bool
        """
        def possible(total, n):
            for i in xrange(1, n//2+1):
                if total*i%n == 0:
                    return True
            return False
        n, s = len(A), sum(A)
        if not possible(n, s):
            return False

        sums = [set() for _ in xrange(n//2+1)]
        sums[0].add(0)
        for num in A:  # O(n) times
            for i in reversed(xrange(1, n//2+1)):  # O(n) times
                for prev in sums[i-1]:  # O(1) + O(2) + ... O(n/2) = O(n^2) times
                    sums[i].add(prev+num)
        for i in xrange(1, n//2+1):
            if s*i%n == 0 and s*i//n in sums[i]:
                return True
        return False

# 0810  Hard  810. Chalkboard XOR Game

In [ ]:
class Solution:
    def xorGame(self, nums):
        xor = 0
        for i in nums: xor ^= i
        return xor == 0 or len(nums) % 2 == 0

# 0810 Hard 810 Chalkboard XOR Game

In [ ]:
# Time:  O(n)
# Space: O(1)

from operator import xor
from functools import reduce


class Solution(object):
    def xorGame(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        return reduce(xor, nums) == 0 or \
            len(nums) % 2 == 0

# 0815  Hard  815. Bus Routes

In [ ]:
class Solution:
    def numBusesToDestination(self, routes, starterBus, targetBus):
        path, travel, travelTaken, used = collections.defaultdict(set), [starterBus], 0, set()
        for i, route in enumerate(routes):
            for bus in route:
                path[bus].add(i)
        while travel:
            new = []
            for bus in travel:
                if bus == targetBus:
                    return travelTaken
                for route in path[bus]:
                    if route not in used:
                        used.add(route)
                        for nextBus in routes[route]:
                            if nextBus != bus:
                                new.append(nextBus)
            travelTaken += 1
            travel = new
        return -1

# 0815 Hard 815 Bus Routes

In [ ]:
# Time:  O(|V| + |E|)
# Space: O(|V| + |E|)

import collections


class Solution(object):
    def numBusesToDestination(self, routes, S, T):
        """
        :type routes: List[List[int]]
        :type S: int
        :type T: int
        :rtype: int
        """
        if S == T:
            return 0

        to_route = collections.defaultdict(set)
        for i, route in enumerate(routes):
            for stop in route:
                to_route[stop].add(i)

        result = 1
        q = [S]
        lookup = set([S])
        while q:
            next_q = []
            for stop in q:
                for i in to_route[stop]:
                    for next_stop in routes[i]:
                        if next_stop in lookup:
                            continue
                        if next_stop == T:
                            return result
                        next_q.append(next_stop)
                        to_route[next_stop].remove(i)
                        lookup.add(next_stop)
            q = next_q
            result += 1

        return -1

# 0818  Hard  818. Race Car

In [ ]:
class Solution:
    def racecar(self, target):
        q, cnt, used = [(0, 1)], 0, {(0, 1)}
        while q:
            new = []
            for pos, speed in q:
                if pos == target:
                    return cnt
                elif pos > 20000 or -20000 > pos:
                    continue
                if (pos + speed, speed * 2) not in used:
                    new.append((pos + speed, speed * 2))
                    used.add((pos + speed, speed * 2))
                if speed > 0 and (pos, -1) not in used:
                    new.append((pos, -1))
                    used.add((pos, -1))
                elif speed < 0 and (pos, 1) not in used:
                    new.append((pos, 1))
                    used.add((pos, 1))
            q = new
            cnt += 1

# 0818 Hard 818 Race Car

In [ ]:
# Time : O(nlogn), n is the value of the target
# Space: O(n)


class Solution(object):
    def racecar(self, target):
        dp = [0] * (target+1)
        for i in xrange(1, target+1):
            # 2^(k-1) <= i < 2^k
            k = i.bit_length()

            # case 1. drive exactly i at best
            #         seq(i) = A^k
            if i == 2**k-1:
                dp[i] = k
                continue

            # case 2. drive cross i at 2^k-1, and turn back to i
            #         seq(i) = A^k -> R -> seq(2^k-1 - i)
            dp[i] = k+1 + dp[2**k-1 - i]

            # case 3. drive less then 2^k-1, and turn back some distance,
            #         and turn back again to make the direction is the same
            #         seq(i) = shortest(seq(i), A^(k-1) -> R -> A^j -> R ->
            #                                   seq(i - (2^(k-1)-1) + (2^j-1)),
            #                  where 0 <= j < k-1)
            #         => dp[i] = min(dp[i], (k-1) + 1 + j + 1 +
            #                               dp[i - (2**(k-1)-1) + (2**j-1)])
            for j in xrange(k-1):
                dp[i] = min(dp[i], k+j+1 + dp[i - 2**(k-1) + 2**j])

        return dp[-1]

# 0827  Hard  827. Making A Large Island

In [ ]:
class Solution:
    def largestIsland(self, grid):
        def explore(i, j):
            dic[(i, j)], count[curr] = curr, count[curr] + 1
            if i > 0 and grid[i - 1][j] == 1 and (i - 1, j) not in dic: explore(i - 1, j)
            if j > 0 and grid[i][j - 1] == 1 and (i, j - 1) not in dic: explore(i, j - 1)
            if i + 1 < len(grid) and grid[i + 1][j] ==1 and (i + 1, j) not in dic: explore(i + 1, j)
            if j + 1 < len(grid) and grid[i][j + 1] == 1 and (i, j + 1) not in dic: explore(i, j + 1)
        def neighbours(i, j, adj):
            if i > 0 and grid[i - 1][j] == 1 and dic[(i - 1, j)] not in adj: adj.add(dic[(i - 1, j)])
            if j > 0 and grid[i][j - 1] == 1 and dic[(i, j - 1)] not in adj: adj.add(dic[(i, j - 1)])
            if i + 1 < len(grid) and grid[i + 1][j] ==1 and dic[(i + 1, j)] not in adj: adj.add(dic[(i + 1, j)])
            if j + 1 < len(grid) and grid[i][j + 1] == 1 and dic[(i, j + 1)] not in adj: adj.add(dic[(i, j + 1)])
            return adj
        curr, dic, count, res = 0, {}, collections.defaultdict(int), 0
        for i in range(len(grid)):
            for j in range(len(grid)):
                if grid[i][j] == 1 and (i, j) not in dic: curr += 1; explore(i, j)
        for i in range(len(grid)):
            for j in range(len(grid)):
                if grid[i][j] == 1: res = max(res, count[dic[(i, j)]])
                else: res = max(res, sum(count[r] for r in neighbours(i, j, set())) + 1)
        return res

# 0827 Hard 827 Making A Large Island

In [ ]:
"""
For each "island" asign them a group id. Also calculate the group's size.
Iterate all the zeros, update the ans.

Time:O(MN)
Space: O(MN) in the worst case.
"""
class Solution(object):
    def largestIsland(self, grid):
        def isValid(i, j, M, N):
            return 0<=i<M and 0<=j<N
        
        def dfs(i, j, groupId):
            if not isValid(i, j, M, N): return
            if grid[i][j]!=1: return
            grid[i][j] = groupId
            groupIdToSize[groupId] += 1
            for iNext, jNext in [(i+1, j), (i-1, j), (i, j-1), (i, j+1)]:
                dfs(iNext, jNext, groupId)

        M = len(grid)
        N = len(grid[0])
        
        zeros = []
        groupIdToSize = {}
        groupId = 2
        for i in xrange(M):
            for j in xrange(N):
                if grid[i][j]==1:
                    groupIdToSize[groupId] = 0
                    dfs(i, j, groupId)
                    groupId += 1
                elif grid[i][j]==0:
                    zeros.append((i, j))
                    
        ans = max(groupIdToSize.values()) if groupIdToSize else 0
        
        for i, j in zeros:
            neiGroupId = set()
            neiSize = 0
            for iNext, jNext in [(i+1, j), (i-1, j), (i, j-1), (i, j+1)]:
                if not isValid(iNext, jNext, M, N): continue
                if grid[iNext][jNext]>1:
                    neiGroupId.add(grid[iNext][jNext])
            
            for groupId in list(neiGroupId):
                neiSize += groupIdToSize[groupId]
            
            ans = max(ans, 1+neiSize)
        
        return ans

# 0828  Hard  828. Count Unique Characters of All Substrings of a Given String

In [ ]:
class Solution:
    def uniqueLetterString(self, S):
        index = {c: [-1, -1] for c in string.ascii_uppercase}
        res = 0
        for i, c in enumerate(S):
            k, j = index[c]
            res += (i - j) * (j - k)
            index[c] = [j, i]
        for c in index:
            k, j = index[c]
            res += (len(S) - j) * (j - k)
        return res % (10**9 + 7)

# 0828 Hard 828 Count Unique Characters of All Substrings of a Given String

In [ ]:
class Solution(object):
    def uniqueLetterString(self, s):
        index = {}
        ans = 0
        for c in 'abcdefghijklmnopqrstuvwxyz':
            index[c.upper()] = [-1, -1]
        
        # count the substring that s[j] is the unique letter
        for k, c in enumerate(s):
            i, j = index[c]
            ans += (j-i) * (k-j)
            index[c] = [j, k]
        
        # count the substring that s[j] is the unique letter, because last iteration did not count the last letter
        for c in index:
            i, j = index[c]
            ans += (j-i) * (len(s)-j)
        return ans

# 0829  Hard  829. Consecutive Numbers Sum

In [ ]:
class Solution:
    def consecutiveNumbersSum(self, N):
        cnt=0
        for d in range(1, N+1):
            diff=d*(d-1)//2
            nd = N - diff
            if nd<=0: break
            if nd%d==0:
                cnt+=1
        return cnt

# 0829 Hard 829 Consecutive Numbers Sum

In [ ]:
# Sliding window
class Solution(object):
    def consecutiveNumbersSum(self, N):
        i = 1
        j = 2
        s = 1 #sum(range(i, j))
        ans = 0
        
        while i<=j and j<=N+1:
            if s==N:
                ans += 1
                s += j
                j += 1
            elif s>N:
                s -= i
                i += 1
            else:
                s += j
                j += 1
        return ans


#Math
class Solution(object):
    def consecutiveNumbersSum(self, N):
        ans = 0
        upperLimit = int((2 * N + 0.25)**0.5 - 0.5) + 2
        
        for x in xrange(1, upperLimit):
            if x%2==0:
                if float(N)/x-N/x==0.5:
                    upper = N/x + x/2
                    lower = N/x - x/2 + 1
                    if 1<=lower and upper<=N:
                        ans += 1
            else:
                if N%x==0:
                    upper = N/x + (x-1)/2
                    lower = N/x - (x-1)/2
                    if 1<=lower and upper<=N: 
                        ans += 1
        return ans

# 0834  Hard  834. Sum of Distances in Tree

In [ ]:
class Solution:
    def sumOfDistancesInTree(self, N, edges):
        tree = collections.defaultdict(set)
        res = [0] * N
        count = [1] * N
        for i, j in edges:
            tree[i].add(j)
            tree[j].add(i)

        def dfs(root, pre):
            for i in tree[root]:
                if i != pre:
                    dfs(i, root)
                    count[root] += count[i]
                    res[root] += res[i] + count[i]

        def dfs2(root, pre):
            for i in tree[root]:
                if i != pre:
                    res[i] = res[root] - count[i] + N - count[i]
                    dfs2(i, root)
        dfs(0, -1)
        dfs2(0, -1)
        return res

# 0834 Hard 834 Sum of Distances in Tree

In [ ]:
# Time:  O(n)
# Space: O(n)

import collections


class Solution(object):
    def sumOfDistancesInTree(self, N, edges):
        """
        :type N: int
        :type edges: List[List[int]]
        :rtype: List[int]
        """
        def dfs(graph, node, parent, count, result):
            for nei in graph[node]:
                if nei != parent:
                    dfs(graph, nei, node, count, result)
                    count[node] += count[nei]
                    result[node] += result[nei]+count[nei]

        def dfs2(graph, node, parent, count, result):
            for nei in graph[node]:
                if nei != parent:
                    result[nei] = result[node]-count[nei] + \
                                  len(count)-count[nei]
                    dfs2(graph, nei, node, count, result)

        graph = collections.defaultdict(list)
        for u, v in edges:
            graph[u].append(v)
            graph[v].append(u)

        count = [1] * N
        result = [0] * N

        dfs(graph, 0, None, count, result)
        dfs2(graph, 0, None, count, result)
        return result

# 0839  Hard  839. Similar String Groups

In [ ]:
class Solution:
    def numSimilarGroups(self, A):
        def explore(s):
            visited.add(s)
            for v in edges[s]:
                if v not in visited: explore(v)
        res, edges, visited = 0, {}, set()
        if len(A) >= 2 * len(A[0]):
            strs = set(A)
            for s in A:
                if s not in edges: edges[s] = set()
                for i in range(len(s) - 1):
                    for j in range(i + 1, len(s)):
                        new = s[:i] + s[j] + s[i + 1:j] + s[i] + s[j + 1:]
                        if new in strs:
                            edges[s].add(new)
                            if new in edges: edges[new].add(s)
                            else: edges[new] = {s}
        else:
            for s in A:
                if s not in edges: edges[s] = set()
                for t in A:
                    if s != t:
                        same = 0
                        for i, c in enumerate(t):
                            if c == s[i]: same += 1
                        if same == len(s) - 2: 
                            edges[s].add(t)
                            if t in edges: edges[t].add(s)
                            else: edges[t] = {s}
        for s in A:
            if s not in visited:
                res += 1
                explore(s)
        return res              

# 0839 Hard 839 Similar String Groups

In [ ]:
# Time:  O(n^2 * l) ~ O(n * l^4)
# Space: O(n) ~ O(n * l^3)

import collections
import itertools


class UnionFind(object):
    def __init__(self, n):
        self.set = range(n)
        self.__size = n

    def find_set(self, x):
        if self.set[x] != x:
            self.set[x] = self.find_set(self.set[x])  # path compression.
        return self.set[x]

    def union_set(self, x, y):
        x_root, y_root = map(self.find_set, (x, y))
        if x_root == y_root:
            return False
        self.set[min(x_root, y_root)] = max(x_root, y_root)
        self.__size -= 1
        return True

    def size(self):
        return self.__size


class Solution(object):
    def numSimilarGroups(self, A):
        def isSimilar(a, b):
            diff = 0
            for x, y in itertools.izip(a, b):
                if x != y:
                    diff += 1
                    if diff > 2:
                        return False
            return diff == 2

        N, L = len(A), len(A[0])
        union_find = UnionFind(N)
        if N < L*L:
            for (i1, word1), (i2, word2) in \
                    itertools.combinations(enumerate(A), 2):
                if isSimilar(word1, word2):
                    union_find.union_set(i1, i2)
        else:
            buckets = collections.defaultdict(list)
            lookup = set()
            for i in xrange(len(A)):
                word = list(A[i])
                if A[i] not in lookup:
                    buckets[A[i]].append(i)
                    lookup.add(A[i])
                for j1, j2 in itertools.combinations(xrange(L), 2):
                    word[j1], word[j2] = word[j2], word[j1]
                    buckets["".join(word)].append(i)
                    word[j1], word[j2] = word[j2], word[j1]
            for word in A:  # Time:  O(n * l^4)
                for i1, i2 in itertools.combinations(buckets[word], 2):
                    union_find.union_set(i1, i2)
        return union_find.size()

# 0843  Hard  843. Guess the Word

In [ ]:
# """
# This is Master's API interface.
# You should not implement it, or speculate about its implementation
# """
#class Master:
#    def guess(self, word):
#        """
#        :type word: str
#        :rtype int
#        """

class Solution:
    def findSecretWord(self, wordlist, master):
        n = 0
        while n < 6:
            count = collections.Counter(w1 for w1, w2 in itertools.permutations(wordlist, 2) if sum(i == j for i, j in zip(w1, w2)) == 0)
            guess = min(wordlist, key = lambda w: count[w])
            n = master.guess(guess)
            wordlist = [w for w in wordlist if sum(i == j for i, j in zip(w, guess)) == n]

# 0843 Hard 843 Guess the Word

In [ ]:
# """
# This is Master's API interface.
# You should not implement it, or speculate about its implementation
# """
#class Master(object):
#    def guess(self, word):
#        """
#        :type word: str
#        :rtype int
#        """

class Solution(object):
    def findSecretWord(self, wordlist, master):
        def similarity(w1, w2):
            count = 0
            for i in xrange(6):
                if w1[i]==w2[i]: count += 1
            return count
        
        for _ in xrange(10):
            
            temp = []
            
            word = random.choice(wordlist)
            k = master.guess(word)
            if k==6: return
            
            for otherWord in wordlist:
                if otherWord==word: continue
                if similarity(word, otherWord)==k: temp.append(otherWord)
            
            wordlist = temp

# 0847  Hard  847. Shortest Path Visiting All Nodes

In [ ]:
class Solution:
    def shortestPathLength(self, graph):
        memo, final, q = set(), (1 << len(graph)) - 1, collections.deque([(i, 0, 1 << i) for i in range(len(graph))])
        while q:
            node, steps, state = q.popleft()
            if state == final: return steps
            for v in graph[node]:
                if (state | 1 << v, v) not in memo:
                    q.append((v, steps + 1, state | 1 << v))
                    memo.add((state | 1 << v, v))

# 0847 Hard 847 Shortest Path Visiting All Nodes

In [ ]:
# Time:  O(n * 2^n)
# Space: O(n * 2^n)

import collections


class Solution(object):
    def shortestPathLength(self, graph):
        """
        :type graph: List[List[int]]
        :rtype: int
        """
        dp = [[float("inf")]*(len(graph))
              for _ in xrange(1 << len(graph))]
        q = collections.deque()
        for i in xrange(len(graph)):
            dp[1 << i][i] = 0
            q.append((1 << i, i))
        while q:
            state, node = q.popleft()
            steps = dp[state][node]
            for nei in graph[node]:
                new_state = state | (1 << nei)
                if dp[new_state][nei] == float("inf"):
                    dp[new_state][nei] = steps+1
                    q.append((new_state, nei))
        return min(dp[-1])

# 0850  Hard  850. Rectangle Area II

In [ ]:
class Solution:
    def rectangleArea(self, rectangles):
        xs = sorted(set([x for x1, y1, x2, y2 in rectangles for x in [x1, x2]] + [0]))
        x_i = {v: i for i, v in enumerate(xs)}
        count = [0] * len(x_i)
        L = []
        for x1, y1, x2, y2 in rectangles:
            L.append([y1, x1, x2, 1])
            L.append([y2, x1, x2, -1])
        L.sort()
        cur_y = cur_x_sum = area = 0
        for y, x1, x2, sig in L:
            area += (y - cur_y) * cur_x_sum
            cur_y = y
            for i in range(x_i[x1], x_i[x2]):
                count[i] += sig
            cur_x_sum = sum(x2 - x1 if c else 0 for x1, x2, c in zip(xs, xs[1:], count))
        return area % (10 ** 9 + 7)

# 0850 Hard 850 Rectangle Area II

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

class SegmentTreeNode(object):
    def __init__(self, start, end):
        self.start, self.end = start, end
        self.total = self.count = 0
        self._left = self._right = None

    def mid(self):
        return (self.start+self.end) // 2

    def left(self):
        self._left = self._left or SegmentTreeNode(self.start, self.mid())
        return self._left

    def right(self):
        self._right = self._right or SegmentTreeNode(self.mid(), self.end)
        return self._right

    def update(self, X, i, j, val):
        if i >= j:
            return 0
        if self.start == i and self.end == j:
            self.count += val
        else:
            self.left().update(X, i, min(self.mid(), j), val)
            self.right().update(X, max(self.mid(), i), j, val)
        if self.count > 0:
            self.total = X[self.end]-X[self.start]
        else:
            self.total = self.left().total + self.right().total
        return self.total


class Solution(object):
    def rectangleArea(self, rectangles):
        """
        :type rectangles: List[List[int]]
        :rtype: int
        """
        OPEN, CLOSE = 1, -1
        events = []
        X = set()
        for x1, y1, x2, y2 in rectangles:
            events.append((y1, OPEN, x1, x2))
            events.append((y2, CLOSE, x1, x2))
            X.add(x1)
            X.add(x2)
        events.sort()
        X = sorted(X)
        Xi = {x: i for i, x in enumerate(X)}

        st = SegmentTreeNode(0, len(X)-1)
        result = 0
        cur_x_sum = 0
        cur_y = events[0][0]
        for y, typ, x1, x2 in events:
            result += cur_x_sum * (y-cur_y)
            cur_x_sum = st.update(X, Xi[x1], Xi[x2], typ)
            cur_y = y
        return result % (10**9+7)

# 0854  Hard  854. K Similar Strings

In [ ]:
class Solution:
    def kSimilarity(self, A, B):
        b, n, k, stack = [c for c in B], len(A), float("inf"), [(0, 0, [c for c in A])]
        while stack:
            i, cnt, s = stack.pop()
            while i < n and s[i] == b[i]:
                i += 1
            if i == n:
                if cnt < k:
                    k = cnt
            else:
                for j in range(i + 1, n):
                    if s[j] == b[i] and s[j] != b[j]:
                        ls = s[:]
                        ls[i], ls[j] = ls[j], ls[i]
                        stack.append((i + 1, cnt + 1, ls))
        return k

# 0857  Hard  857. Minimum Cost to Hire K Workers

In [ ]:
class Solution:
    def mincostToHireWorkers(self, quality, wage, K):
        workers, res, heap, sumq = sorted((w / q, q, w) for q, w in zip(quality, wage)), float("inf"), [], 0
        for ratio, q, w in workers:
            heapq.heappush(heap, -q)
            sumq += q
            if len(heap) > K:
                sumq += heapq.heappop(heap)
            if len(heap) == K:
                res = min(res, ratio * sumq)
        return res

# 0857 Hard 857 Minimum Cost to Hire K Workers

In [ ]:
"""
Time: O(NLogN)
Space: O(N)

1. In the paidGroup, one of the worker will be paid by his minimum wage.
Otherwise, the cost can be lower. Let's call this person "captain".
"captain" is getting paid "wage".

2. People in the paidGroup will be paid according to quality.
So people in the paidGroup is getting paid (individual quality) * (captain's wage/quality)
The total cost will be (sum of the quality in the paidGroup) * (captain's wage/quality).
So given a captain, we want the quality in the paidGroup as low as possible to have minimum cost.

3. Declare a list of worker's info (ratio, quality, wage).
Sort the list. Sorting the list is important.
In the next step, we are going to iterate over the workers and assume they are "captain".
This make sure that we can afford to pay worker already in the paidGroup larger than their minimum wage.

4. Iterate over the workers and assume they are "captain".
Also adding the worker to the paidGroup.
The paidGroup will at most have k people. With lowest quality. Using a max heap to maintain.
Also, tracking the total quality in the paidGroup (sumQ), so we can calculate the cost when `len(paidGroup)==k`.
Return the minimum cost.
"""
class Solution(object):
    def mincostToHireWorkers(self, quality, wage, k):
        cost = float('inf')
        
        workers = [(wage[i]/float(quality[i]), quality[i], wage[i]) for i in xrange(len(quality))]
        workers = sorted(workers)
        
        paidGroup = []
        sumQ = 0
        
        for r, q, w in workers:
            heapq.heappush(paidGroup, -q)
            sumQ += q
            
            if len(paidGroup)>k:
                sumQ -= -heapq.heappop(paidGroup)
            
            if len(paidGroup)==k:
                cost = min(cost, sumQ*r)
        
        return cost

# 0862  Hard  862. Shortest Subarray with Sum at Least K

In [ ]:
class Solution:
    def shortestSubarray(self, A, K):
        heap, l, sm = [], float("inf"), 0
        heapq.heappush(heap, (0, -1))
        for i, num in enumerate(A):
            sm += num
            dif = sm - K
            while heap and (heap[0][0] <= dif or i - heap[0][1] >= l):
                preSum, preIndex = heapq.heappop(heap)
                if i - preIndex < l:
                    l = i - preIndex
            heapq.heappush(heap, (sm, i))
        return l < float("inf") and l or -1

# 0862 Hard 862 Shortest Subarray with Sum at Least K

In [ ]:
# Time:  O(n)
# Space: O(n)

import collections


class Solution(object):
    def shortestSubarray(self, A, K):
        """
        :type A: List[int]
        :type K: int
        :rtype: int
        """
        accumulated_sum = [0]*(len(A)+1)
        for i in xrange(len(A)):
            accumulated_sum[i+1] = accumulated_sum[i]+A[i]

        result = float("inf")
        mono_increasing_q = collections.deque()
        for i, curr in enumerate(accumulated_sum):
            while mono_increasing_q and curr <= \
                    accumulated_sum[mono_increasing_q[-1]]:
                mono_increasing_q.pop()
            while mono_increasing_q and \
                    curr-accumulated_sum[mono_increasing_q[0]] >= K:
                result = min(result, i-mono_increasing_q.popleft())
            mono_increasing_q.append(i)
        return result if result != float("inf") else -1

# 0864  Hard  864. Shortest Path to Get All Keys

In [ ]:
class Solution:
    def shortestPathAllKeys(self, grid):
        final, m, n, si, sj = 0, len(grid), len(grid[0]), 0, 0
        for i in range(m):
            for j in range(n):
                if grid[i][j] in "abcdef":
                    final |= 1 << ord(grid[i][j]) - ord("a")
                elif grid[i][j] == "@":
                    si, sj = i, j
        q, memo = [(0, si, sj, 0)], set()
        while q:
            moves, i, j, state = heapq.heappop(q)
            if state == final: return moves
            for x, y in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)):
                if 0 <= x < m and 0 <= y < n and grid[x][y] != "#":
                    if grid[x][y].isupper() and not state & 1 << (ord(grid[x][y].lower()) - ord("a")): continue
                    newState = ord(grid[x][y]) >= ord("a") and state | 1 << (ord(grid[x][y]) - ord("a")) or state
                    if (newState, x, y) not in memo:
                        memo.add((newState, x, y))
                        heapq.heappush(q, (moves + 1, x, y, newState))
        return -1

# 0864 Hard 864 Shortest Path to Get All Keys

In [ ]:
# Time:  O(k*r*c + |E|log|V|) = O(k*r*c + (k*|V|)*log|V|)
#                             = O(k*r*c + (k*(k*2^k))*log(k*2^k))
#                             = O(k*r*c + (k*(k*2^k))*(logk + k*log2))
#                             = O(k*r*c + (k*(k*2^k))*k)
#                             = O(k*r*c + k^3*2^k)
# Space: O(|V|) = O(k*2^k)

import collections
import heapq


class Solution(object):
    def shortestPathAllKeys(self, grid):
        """
        :type grid: List[str]
        :rtype: int
        """
        directions = [(0, -1), (0, 1), (-1, 0), (1, 0)]

        def bfs(grid, source, locations):
            r, c = locations[source]
            lookup = [[False]*(len(grid[0])) for _ in xrange(len(grid))]
            lookup[r][c] = True
            q = collections.deque([(r, c, 0)])
            dist = {}
            while q:
                r, c, d = q.popleft()
                if source != grid[r][c] != '.':
                    dist[grid[r][c]] = d
                    continue
                for direction in directions:
                    cr, cc = r+direction[0], c+direction[1]
                    if not ((0 <= cr < len(grid)) and
                            (0 <= cc < len(grid[cr]))):
                        continue
                    if grid[cr][cc] != '#' and not lookup[cr][cc]:
                        lookup[cr][cc] = True
                        q.append((cr, cc, d+1))
            return dist

        locations = {place: (r, c)
                     for r, row in enumerate(grid)
                     for c, place in enumerate(row)
                     if place not in '.#'}
        dists = {place: bfs(grid, place, locations) for place in locations}

        # Dijkstra's algorithm
        min_heap = [(0, '@', 0)]
        best = collections.defaultdict(lambda: collections.defaultdict(
                                                   lambda: float("inf")))
        best['@'][0] = 0
        target_state = 2**sum(place.islower() for place in locations)-1
        while min_heap:
            cur_d, place, state = heapq.heappop(min_heap)
            if best[place][state] < cur_d:
                continue
            if state == target_state:
                return cur_d
            for dest, d in dists[place].iteritems():
                next_state = state
                if dest.islower():
                    next_state |= (1 << (ord(dest)-ord('a')))
                elif dest.isupper():
                    if not (state & (1 << (ord(dest)-ord('A')))):
                        continue
                if cur_d+d < best[dest][next_state]:
                    best[dest][next_state] = cur_d+d
                    heapq.heappush(min_heap, (cur_d+d, dest, next_state))
        return -1

# 0871  Hard  871. Minimum Number of Refueling Stops

In [ ]:
class Solution:
    def minRefuelStops(self, target, startFuel, stations):
        q, n, memo = [(0, -startFuel, 0, 0)], len(stations), set()
        while q:
            refill, fuel, pos, index = heapq.heappop(q)
            fuel *= -1
            if index == n:
                if fuel - (target - pos) >= 0:
                    return refill
            else:
                sPos, add = stations[index]
                if (index, refill) not in memo and fuel - (sPos - pos) >= 0:
                    memo.add((index, refill))
                    f1 = (fuel - (sPos - pos) + add) * -1
                    f2 = (fuel - (sPos - pos)) * -1
                    heapq.heappush(q, (refill + 1, f1, sPos, index + 1))
                    heapq.heappush(q, (refill, f2, sPos, index + 1))
        return -1

# 0871 Hard 871 Minimum Number of Refueling Stops

In [ ]:
# Time:  O(nlogn)
# Space: O(n)

import heapq


class Solution(object):
    def minRefuelStops(self, target, startFuel, stations):
        """
        :type target: int
        :type startFuel: int
        :type stations: List[List[int]]
        :rtype: int
        """
        max_heap = []
        stations.append((target, float("inf")))

        result = prev = 0
        for location, capacity in stations:
            startFuel -= location - prev
            while max_heap and startFuel < 0:
                startFuel += -heapq.heappop(max_heap)
                result += 1
            if startFuel < 0:
                return -1
            heapq.heappush(max_heap, -capacity)
            prev = location

        return result

# 0878  Hard  878. Nth Magical Number

In [ ]:
class Solution:
    
    def gcd(self, a, b):
        while b:
            a, b = b, a % b
        return a
    
    def count(self, num, A, B, C):
        return num // A + num // B - num // C
    
    def nthMagicalNumber(self, N, A, B):
        l, r, C = 2, 2 ** 63  - 1, A * B // self.gcd(A, B)
        while l < r:
            mid = (l + r) // 2
            if self.count(mid, A, B, C) < N:
                l = mid + 1
            else:
                r = mid
        return l % (10 ** 9 + 7)

# 0878 Hard 878 Nth Magical Number

In [ ]:
# Time:  O(logn)
# Space: O(1)

class Solution(object):
    def nthMagicalNumber(self, N, A, B):
        """
        :type N: int
        :type A: int
        :type B: int
        :rtype: int
        """
        def gcd(a, b):
            while b:
                a, b = b, a % b
            return a

        def check(A, B, N, lcm, target):
            return target//A + target//B - target//lcm >= N

        lcm = A*B // gcd(A, B)
        left, right = min(A, B), max(A, B)*N
        while left <= right:
            mid = left + (right-left)//2
            if check(A, B, N, lcm, mid):
                right = mid-1
            else:
                left = mid+1
        return left % (10**9 + 7)

# 0879  Hard  879. Profitable Schemes

In [ ]:
class Solution:
    def profitableSchemes(self, G: int, P: int, group: List[int], profit: List[int]) -> int:
        dp = [[0] * (G + 1) for i in range(P + 1)]
        dp[0][0] = 1
        for p, g in zip(profit, group):
            for i in range(P, -1, -1):
                for j in range(G - g, -1, -1):
                    dp[min(i + p, P)][j + g] += dp[i][j]
        return sum(dp[P]) % (10**9 + 7)

# 0879 Hard 879 Profitable Schemes

In [ ]:
"""
TLE
dp[i][n][p] := considering profit[:i], what is the number of ways produce profit p with n people.
"""
class Solution(object):
    def profitableSchemes(self, maxMember, minProfit, group, profit):
        P = sum(profit)
        N = sum(group)
        dp = [[[0 for _ in xrange(P+1)] for _ in xrange(N+1)] for _ in xrange(len(profit)+1)]
        dp[0][0][0] = 1
        
        count = 0
        for i in xrange(1, len(profit)+1):
            for n in xrange(N+1):
                for p in xrange(P+1):
                    dp[i][n][p] = (dp[i-1][n-group[i-1]][p-profit[i-1]] if p-profit[i-1]>=0 and n-group[i-1]>=0 else 0) + dp[i-1][n][p]
                    if i==len(profit) and p>=minProfit and n<=maxMember: count += dp[i][n][p]
        return count



"""
dp[i][g][p] := consider only crime[:i] the scheme that can generate profit p using man power g.
"""
class Solution(object):
    def profitableSchemes(self, n, minProfit, group, profit):
        N = len(profit)
        
        dp = [[[0]*(n+2) for _ in xrange(minProfit+1)] for _ in xrange(N+1)]
        dp[0][0][0] = 1
        
        for i in xrange(1, N+1):
            for p in xrange(minProfit+1):
                for g in xrange(n+1):
                    pi = profit[i-1]
                    gi = group[i-1]
                    
                    #considerting last round using p and g
                    dp[i][p][g] += dp[i-1][p][g]
                    dp[i][min(pi+p, minProfit)][min(gi+g, n+1)] += dp[i-1][p][g]
        
        ans = 0
        for g in xrange(n+1):
            ans += dp[N][minProfit][g]
        return ans % (10**9 + 7)

# 0882  Hard  882. Reachable Nodes In Subdivided Graph

In [ ]:
class Solution:
    def reachableNodes(self, edges, M, N):
        adj, seen = collections.defaultdict(dict), set()
        for a, b, l in edges:
            adj[a][b] = [l, 0]
            adj[b][a] = [l, 0]
        q = [(0, M, None)]
        while q:
            new = []
            for i, moves, pre in q:
                seen.add(i)
                for j in adj[i]:
                    if moves > adj[i][j][1]:
                        adj[i][j][1] = moves
                        if moves > adj[i][j][0] and j != pre:
                            new.append((j, moves - adj[i][j][0] - 1, i))
            q = new 
        return sum(min(adj[i][j][1] + adj[j][i][1], l) for i, j, l in edges) + len(seen)

# 0882 Hard 882 Reachable Nodes In Subdivided Graph

In [ ]:
# Time:  O((|E| + |V|) * log|V|) = O(|E| * log|V|),
#        if we can further to use Fibonacci heap, it would be O(|E| + |V| * log|V|)
# Space: O(|E| + |V|) = O(|E|)

import collections
import heapq

class Solution(object):
    def reachableNodes(self, edges, M, N):
        """
        :type edges: List[List[int]]
        :type M: int
        :type N: int
        :rtype: int
        """
        adj = [[] for _ in xrange(N)]
        for u, v, w in edges:
            adj[u].append((v, w))
            adj[v].append((u, w))

        min_heap = [(0, 0)]
        best = collections.defaultdict(lambda: float("inf"))
        best[0] = 0
        count = collections.defaultdict(lambda: collections.defaultdict(int))
        result = 0
        while min_heap:
            curr_total, u = heapq.heappop(min_heap)  # O(|V|*log|V|) in total
            if best[u] < curr_total:
                continue
            result += 1
            for v, w in adj[u]:
                count[u][v] = min(w, M-curr_total)
                next_total = curr_total+w+1
                if next_total <= M and next_total < best[v]:
                    best[v] = next_total
                    heapq.heappush(min_heap, (next_total, v))  # binary heap O(|E|*log|V|) in total
                                                               # Fibonacci heap O(|E|) in total
        for u, v, w in edges:
            result += min(w, count[u][v]+count[v][u])
        return result

# 0887  Hard  887. Super Egg Drop

In [ ]:
class Solution:
    def superEggDrop(self, K, N):
        drops = 0                           # the number of eggs dropped
        floors = [0 for _ in range(K + 1)]  # floors[i] is the number of floors that can be checked with i eggs

        while floors[K] < N:                # until we can reach N floors with K eggs 

            for eggs in range(K, 0, -1):
                floors[eggs] += 1 + floors[eggs - 1]
            drops += 1

        return drops

# 0887 Hard 887 Super Egg Drop

In [ ]:
# Time:  O(klogn)
# Space: O(1)

class Solution(object):
    def superEggDrop(self, K, N):
        """
        :type K: int
        :type N: int
        :rtype: int
        """
        def check(n, K, N):
            # let f(n, K) be the max number of floors could be solved by n moves and K eggs,
            # we want to do binary search to find min of n, s.t. f(n, K) >= N,
            # if we use one move to drop egg with X floors
            # 1. if it breaks, we can search new X in the range [X+1, X+f(n-1, K-1)]
            # 2. if it doesn't break, we can search new X in the range [X-f(n-1, K), X-1]
            # => f(n, K) = (X+f(n-1, K-1))-(X-f(n-1, K))+1 = f(n-1, K-1)+f(n-1, K)+1
            # => (1) f(n, K)   = f(n-1, K)  +1+f(n-1, K-1)
            #    (2) f(n, K-1) = f(n-1, K-1)+1+f(n-1, K-2)
            # let g(n, K) = f(n, K)-f(n, K-1), and we subtract (1) by (2)
            # => g(n, K) = g(n-1, K)+g(n-1, K-1), obviously, it is binomial coefficient
            # => C(n, K) = g(n, K) = f(n, K)-f(n, K-1),
            #    which also implies if we have one more egg with n moves and x-1 egges, we can have more C(n, x) floors solvable
            # => f(n, K) = C(n, K)+f(n, K-1) = C(n, K) + C(n, K-1) + ... + C(n, 1) + f(n, 0) = sum(C(n, k) for k in [1, K])
            # => all we have to do is to check sum(C(n, k) for k in [1, K]) >= N,
            #    if true, there must exist a 1-to-1 mapping from each F in [1, N] to each sucess and failure sequence of every C(n, k) combinations for k in [1, K]
            total, c = 0, 1
            for k in xrange(1, K+1):
                c *= n-k+1
                c //= k
                total += c
                if total >= N:
                    return True
            return False

        left, right = 1, N
        while left <= right:
            mid = left + (right-left)//2
            if check(mid, K, N):
                right = mid-1
            else:
                left = mid+1
        return left

# 0891  Hard  891. Sum of Subsequence Widths

In [ ]:
class Solution:
    def sumSubseqWidths(self, A):
        A.sort()
        res=0
        for i in range(len(A)):
            res*=2
            res-=A[i]
            res+=A[~i]
        return res % (10**9+7)

# 0891 Hard 891 Sum of Subsequence Widths

In [ ]:
# Time:  O(n)
# Spce:  O(1)

class Solution(object):
    def sumSubseqWidths(self, A):
        """
        :type A: List[int]
        :rtype: int
        """
        M = 10**9+7
        # sum(A[i] * (2^i - 2^(len(A)-1-i))), i = 0..len(A)-1
        # <=>
        # sum(((A[i] - A[len(A)-1-i]) * 2^i), i = 0..len(A)-1
        result, c = 0, 1
        A.sort()
        for i in xrange(len(A)):
            result = (result + (A[i]-A[len(A)-1-i])*c % M) % M
            c = (c<<1) % M
        return result

# 0895  Hard  895. Maximum Frequency Stack

In [ ]:
class FreqStack:

    def __init__(self):
        self.stacks = collections.defaultdict(list)
        self.freq = collections.Counter()
        self.maxFreq = 0

    def push(self, x):
        self.freq[x] += 1 
        self.maxFreq = max(self.maxFreq, self.freq[x])
        self.stacks[self.freq[x]].append(x)

    def pop(self):
        num = self.stacks[self.maxFreq].pop()
        self.freq[num] -= 1 
        if not self.stacks[self.maxFreq]: self.maxFreq -= 1
        return num

# 0895 Hard 895 Maximum Frequency Stack

In [ ]:
# Time:  O(1)
# Space: O(n)

import collections


class FreqStack(object):

    def __init__(self):
        self.__freq = collections.Counter()
        self.__group = collections.defaultdict(list)
        self.__maxfreq = 0

    def push(self, x):
        """
        :type x: int
        :rtype: void
        """
        self.__freq[x] += 1
        if self.__freq[x] > self.__maxfreq:
            self.__maxfreq = self.__freq[x]
        self.__group[self.__freq[x]].append(x)   

    def pop(self):
        """
        :rtype: int
        """
        x = self.__group[self.__maxfreq].pop()
        if not self.__group[self.__maxfreq]:
            self.__group.pop(self.__maxfreq)
            self.__maxfreq -= 1
        self.__freq[x] -= 1
        if not self.__freq[x]:
            self.__freq.pop(x)
        return x

# 0899  Hard  899. Orderly Queue

In [ ]:
class Solution:
    def orderlyQueue(self, S, K):
        return "".join(sorted(S)) if K > 1 else min(S[i:] + S[:i] for i in range(len(S)))

# 0899 Hard 899 Orderly Queue

In [ ]:
# Time:  O(n^2)
# Space: O(n)

class Solution(object):
    def orderlyQueue(self, S, K):
        """
        :type S: str
        :type K: int
        :rtype: str
        """
        if K == 1:
            return min(S[i:] + S[:i] for i in xrange(len(S)))
        return "".join(sorted(S))

# 0902  Hard  902. Numbers At Most N Given Digit Set

In [ ]:
class Solution:
    def atMostNGivenDigitSet(self, D, N):
        def less(c):
            return len([char for char in D if char < c])
        d, cnt, l = len(D), 0, len(str(N))
        # For numbers which have less digits than N, simply len(D) ** digits_length different numbers can be created
        for i in range(1, l):
            cnt += d ** i
        """
        We should also consider edge cases where previous digits match with related digits in N. In this case, we can make a number with             previous digits + (digits less than N[i]) + D ** remaining length
        """
        for i, c in enumerate(str(N)):
            cnt += less(c) * (d ** (l - i - 1))
            if c not in D: break
            if i == l - 1: cnt += 1
        return cnt

# 0902 Hard 902 Numbers At Most N Given Digit Set

In [ ]:
# Time:  O(logn)
# Space: O(logn)

class Solution(object):
    def atMostNGivenDigitSet(self, D, N):
        """
        :type D: List[str]
        :type N: int
        :rtype: int
        """
        str_N = str(N)
        set_D = set(D)
        result = sum(len(D)**i for i in xrange(1, len(str_N)))
        i = 0
        while i < len(str_N):
            result += sum(c < str_N[i] for c in D) * (len(D)**(len(str_N)-i-1))
            if str_N[i] not in set_D:
                break
            i += 1
        return result + int(i == len(str_N))

# 0903  Hard  903. Valid Permutations for DI Sequence

In [ ]:
class Solution:
    def numPermsDISequence(self, S: str) -> int:
        dp = [1] * (len(S) + 1)
        for c in S:
            if c == "I":
                dp = dp[:-1]
                for i in range(1, len(dp)):
                    dp[i] += dp[i - 1]
            else:
                dp = dp[1:]
                for i in range(len(dp) - 1)[::-1]:
                    dp[i] += dp[i + 1]
        return dp[0] % (10**9 + 7)

# 0903 Hard 903 Valid Permutations for DI Sequence

In [ ]:
# Time:  O(n^2)
# Space: O(n)

class Solution(object):
    def numPermsDISequence(self, S):
        """
        :type S: str
        :rtype: int
        """
        dp = [1]*(len(S)+1)
        for c in S:
            if c == "I":
                dp = dp[:-1]
                for i in xrange(1, len(dp)):
                    dp[i] += dp[i-1]
            else:
                dp = dp[1:]
                for i in reversed(xrange(len(dp)-1)):
                    dp[i] += dp[i+1]
        return dp[0] % (10**9+7)

# 0906  Hard  906. Super Palindromes

In [ ]:
class Solution:
    def superpalindromesInRange(self, L, R):
        L, R = int(L), int(R)
        left = int(math.floor(math.sqrt(L)))
        right = int(math.ceil(math.sqrt(R)))
        n1, n2 = len(str(left)), len(str(right))
        n1 = n1//2 if n1%2==0 else n1//2+1
        n2 = n2//2 if n2%2==0 else n2//2+1
        start = int('1' + '0'*(n1 - 1))
        end = int('9' * n2) + 1
        ans = 0 
        for i in range(start, end):
            x = str(i)
            num1 = int(x + x[::-1])
            num2 = int(x + x[:-1][::-1])
            for num in [num1, num2]:
                cand = num * num
                if L <= cand <= R and str(cand) == str(cand)[::-1]:
                    ans += 1
        return ans

# 0906 Hard 906 Super Palindromes

In [ ]:
# Time:  O(n^0.25 * logn)
# Space: O(logn)

class Solution(object):
    def superpalindromesInRange(self, L, R):
        """
        :type L: str
        :type R: str
        :rtype: int
        """
        def is_palindrome(k):
            return str(k) == str(k)[::-1]

        K = int((10**((len(R)+1)*0.25)))
        l, r = int(L), int(R)

        result = 0

        # count odd length
        for k in xrange(K):
            s = str(k)
            t = s + s[-2::-1]
            v = int(t)**2
            if v > r:
                break
            if v >= l and is_palindrome(v):
                result += 1

        # count even length
        for k in xrange(K):
            s = str(k)
            t = s + s[::-1]
            v = int(t)**2
            if v > r:
                break
            if v >= l and is_palindrome(v):
                result += 1

        return result

# 0913  Hard  913. Cat and Mouse

In [ ]:
class Solution:
    def catMouseGame(self, graph: 'List[List[int]]') -> 'int':
        mouse_visited = [False] * len(graph)
        mouse_win_map = [[None for column in range(len(graph))] for row in range(len(graph))]
        cat_visited = [False] * len(graph)
        cat_win_map = [[None for column in range(len(graph))] for row in range(len(graph))]
        if self.isMouseWin(graph, 1, 2, mouse_visited, mouse_win_map):
            return 1
        elif self.isCatWin(graph, 1, 2, cat_visited, cat_win_map):
            return 2
        else:
            return 0

    def isMouseWin(self, graph, mouse, cat, mouse_visited, mouse_win_map):
        if mouse == 0:
            return True
        if mouse_win_map[mouse][cat] is not None:
            return mouse_win_map[mouse][cat]
        mouse_visited[mouse] = True
        for mouseMove in graph[mouse]:
            if mouseMove == 0 or (mouseMove not in graph[cat] and  mouseMove != cat):
                if not mouse_visited[mouseMove]:
                    mouseWinFlag = True
                    for catMove in graph[cat]:
                        if catMove != 0 and not self.isMouseWin(graph, mouseMove, catMove, mouse_visited, mouse_win_map):
                            mouseWinFlag = False
                            break
                    if mouseWinFlag:
                        mouse_visited[mouse] = False
                        mouse_win_map[mouse][cat] = True
                        return True
        mouse_visited[mouse] = False
        mouse_win_map[mouse][cat] = False
        return False

    def isCatWin(self, graph, mouse, cat, cat_visited, cat_win_map):
        if mouse == 0:
            return False
        if cat_win_map[mouse][cat] is not None:
            return cat_win_map[mouse][cat]
        cat_visited[cat] = True
        for mouseMove in graph[mouse]:
            if mouseMove == 0 or (mouseMove not in graph[cat] and  mouseMove != cat):
                catWinFlag = True
                for catMove in graph[cat]:
                    if catMove != 0 and not cat_visited[catMove] and not self.isCatWin(graph, mouseMove, catMove,
                                                                                       cat_visited, cat_win_map):
                        catWinFlag = False
                        break
                if not catWinFlag:
                    cat_visited[cat] = False
                    cat_win_map[mouse][cat] = False
                    return False
        cat_visited[cat] = False
        cat_win_map[mouse][cat] = True
        return True

# 0913 Hard 913 Cat and Mouse

In [ ]:
# Time:  O(n^3)
# Space: O(n^2)

import collections


class Solution(object):
    def catMouseGame(self, graph):
        """
        :type graph: List[List[int]]
        :rtype: int
        """
        HOLE, MOUSE_START, CAT_START = range(3)
        DRAW, MOUSE, CAT = range(3)
        def parents(m, c, t):
            if t == CAT:
                for nm in graph[m]:
                    yield nm, c, MOUSE^CAT^t
            else:
                for nc in graph[c]:
                    if nc != HOLE:
                        yield m, nc, MOUSE^CAT^t

        degree = {}
        ignore = set(graph[HOLE])
        for m in xrange(len(graph)):
            for c in xrange(len(graph)):
                degree[m, c, MOUSE] = len(graph[m])
                degree[m, c, CAT] = len(graph[c])-(c in ignore)
        color = collections.defaultdict(int)
        q = collections.deque()
        for i in xrange(len(graph)):
            if i == HOLE:
                continue
            color[HOLE, i, CAT] = MOUSE
            q.append((HOLE, i, CAT, MOUSE))
            for t in [MOUSE, CAT]:
                color[i, i, t] = CAT
                q.append((i, i, t, CAT))
        while q:
            i, j, t, c = q.popleft()
            for ni, nj, nt in parents(i, j, t):
                if color[ni, nj, nt] != DRAW:
                    continue
                if nt == c:
                    color[ni, nj, nt] = c
                    q.append((ni, nj, nt, c))
                    continue
                degree[ni, nj, nt] -= 1
                if not degree[ni, nj, nt]:
                    color[ni, nj, nt] = c
                    q.append((ni, nj, nt, c))
        return color[MOUSE_START, CAT_START, MOUSE]

    
# Time:  O(n^3)
# Space: O(n^2)
import collections


class Solution2(object):
    def catMouseGame(self, graph):
        """
        :type graph: List[List[int]]
        :rtype: int
        """
        HOLE, MOUSE_START, CAT_START = range(3)
        DRAW, MOUSE, CAT = range(3)
        def parents(m, c, t):
            if t == CAT:
                for nm in graph[m]:
                    yield nm, c, MOUSE^CAT^t
            else:
                for nc in graph[c]:
                    if nc != HOLE:
                        yield m, nc, MOUSE^CAT^t

        color = collections.defaultdict(int)
        degree = {}
        ignore = set(graph[HOLE])
        for m in xrange(len(graph)):
            for c in xrange(len(graph)):
                degree[m, c, MOUSE] = len(graph[m])
                degree[m, c, CAT] = len(graph[c])-(c in ignore)
        q1 = collections.deque()
        q2 = collections.deque()
        for i in xrange(len(graph)):
            if i == HOLE:
                continue
            color[HOLE, i, CAT] = MOUSE
            q1.append((HOLE, i, CAT))
            for t in [MOUSE, CAT]:
                color[i, i, t] = CAT
                q2.append((i, i, t))
        while q1:
            i, j, t = q1.popleft()
            for ni, nj, nt in parents(i, j, t):
                if color[ni, nj, nt] != DRAW:
                    continue
                if t == CAT:
                    color[ni, nj, nt] = MOUSE
                    q1.append((ni, nj, nt))
                    continue
                degree[ni, nj, nt] -= 1
                if not degree[ni, nj, nt]:
                    color[ni, nj, nt] = MOUSE
                    q1.append((ni, nj, nt))
        while q2:
            i, j, t = q2.popleft()
            for ni, nj, nt in parents(i, j, t):
                if color[ni, nj, nt] != DRAW:
                    continue
                if t == MOUSE:
                    color[ni, nj, nt] = CAT
                    q2.append((ni, nj, nt))
                    continue
                degree[ni, nj, nt] -= 1
                if not degree[ni, nj, nt]:
                    color[ni, nj, nt] = CAT
                    q2.append((ni, nj, nt))
        return color[MOUSE_START, CAT_START, MOUSE]

# 0920  Hard  920. Number of Music Playlists

In [ ]:
from functools import lru_cache

class Solution:
    def numMusicPlaylists(self, N, L, K):
        @lru_cache(None)
        def dp(i, j): return +(j == 0) if not i else (dp(i-1, j-1) * (N-j+1) + dp(i-1, j) * max(j-K, 0)) % (10**9+7)
        return dp(L, N)

# 0920 Hard 920 Number of Music Playlists

In [ ]:
# Time:  O(n * l)
# Space: O(l)

class Solution(object):
    def numMusicPlaylists(self, N, L, K):
        """
        :type N: int
        :type L: int
        :type K: int
        :rtype: int
        """
        M = 10**9+7
        dp = [[0 for _ in xrange(1+L)] for _ in xrange(2)]
        dp[0][0] = dp[1][1] = 1
        for n in xrange(1, N+1):
            dp[n % 2][n] = (dp[(n-1) % 2][n-1] * n) % M
            for l in xrange(n+1, L+1):
                dp[n % 2][l] = ((dp[n % 2][l-1] * max(n-K, 0)) % M + \
                                (dp[(n-1) % 2][l-1] * n) % M) % M
        return dp[N % 2][L]

# 0924  Hard  924. Minimize Malware Spread

In [ ]:
class Solution:
    def minMalwareSpread(self, graph, initial):
        def dfs(i):
            nodes.add(i)
            for j in range(len(graph[i])):
                if graph[i][j] and j not in nodes:
                    dfs(j)
        rank, initial = collections.defaultdict(list), set(initial)
        for node in sorted(initial):
            nodes = set()
            dfs(node)
            if nodes & initial == {node}:
                rank[len(nodes)].append(node)
        return rank[max(rank)][0] if rank else min(initial)

# 0924 Hard 924 Minimize Malware Spread

In [ ]:
"""
Lets start from a node in the initial. DFS through the graph. [0]
When a node is infected, we paint it by color1.
After the DFS is done, we start from another node in the initial. DFS through the graph.
When a node is infected, we paint it by color2.
...

We don't paint the node that we already colored. [1]
The more node that are colored by an initial node the more affective it will minimize the malware. [2]
But if a color have two or more initial node, they won't make any difference if taken away. [3]
Because those node are still going to be infected by one another initial node.

So our goal here is to find the initial node that paint the most.
But did not paint other intial node.

I use 'color_data' to store the result [4]
{
    color1: [
        [intial nodes in this color],
        the number of node in this color
    ],
    color2: [
        ...
    ],
    color3: [
        ...
    ],
    ...
}

By 'color_data' I can easily see the things that I care about and calculate the answer
1. The intial nodes in this color
2. The number of node in this color

The time complexity is O(I*N), 
because we loop through the initial nodes.
And each node, it could potential travel all the nodes.
I is the initial nodes count, N is the nodes count.

Space complexity is O(N), because we use colored to keep track of all the nodes.
"""


class Solution(object):
    def minMalwareSpread(self, graph, initial):
        colored = set()
        initial_set = set(initial)
        color_data = {} #[4]
        color = 0
        
        def dfs(node, c):
            stack = [node]
            while stack:
                n = stack.pop()
                if n in colored: continue #[1]
                colored.add(n)

                if n in initial_set:
                    color_data[c][0].append(n)
                color_data[c][1]+=1

                for nb in xrange(len(graph)):
                    if graph[n][nb]==1:
                        stack.append(nb)
                
        # [0]
        for node in initial:
            if color not in color_data:
                color_data[color] = [[], 0]

            dfs(node, color)
            color+=1

        ans = min(initial)
        max_infected = float('-inf')
        for c in color_data.keys():
            if len(color_data[c][0])!=1: continue #[3]
            n = color_data[c][0][0]
            infected = color_data[c][1]

            if color_data[c][1]>max_infected: #[2]
                max_infected = infected
                ans = n
            elif infected==max_infected and n<ans:
                ans = n

        return ans

# 0927  Hard  927. Three Equal Parts

In [ ]:
class Solution(object):
    def threeEqualParts(self, A):
        sm = sum(A)
        if sm % 3: return [-1, -1]
        t = sm // 3
        if not t: return [0, len(A) - 1]
        breaks = [0] + [i for i, x in enumerate(A) if x]
        i1, j1, i2, j2, i3, j3 = breaks[1], breaks[t], breaks[t + 1], breaks[2 * t], breaks[2 * t + 1], breaks[3 * t]
        if not (A[i1: j1 + 1] == A[i2: j2 + 1] == A[i3: j3 + 1]): return [-1, -1]
        if i2 - j1 < len(A) - j3 or i3 - j2 < len(A) - j3: return [-1, -1]
        return [j1 + len(A) - j3 - 1, j2+ len(A) - j3]